In [ ]:
dim_date

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import sequence, to_date, explode, dayofmonth, month, year, weekofyear, dayofweek, quarter, date_format, lit, expr

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Generate daily calendar
    df_calendar = (
        spark.range(1)
             .select(sequence(to_date(lit("2015-01-01")), to_date(lit("2026-12-31")), expr("interval 1 day")).alias("dates"))
             .select(explode("dates").alias("date"))
    )

In [ ]:
# Build dimension table with SK and attributes
    df_dim_date = (
        df_calendar
          .withColumn("sk_date",    expr("year(date)*10000 + month(date)*100 + day(date)"))
          .withColumn("day",        dayofmonth("date"))
          .withColumn("month",      month("date"))
          .withColumn("quarter",    quarter("date"))
          .withColumn("year",       year("date"))
          .withColumn("weekday",    dayofweek("date"))
          .withColumn("week_of_yr", weekofyear("date"))
          .withColumn("month_name", date_format("date", "MMMM"))
          .withColumn("day_name",   date_format("date", "EEEE"))
    )

In [ ]:
# Save to Silver layer
    df_dim_date.write \
        .format("delta") \
        .mode("overwrite") \
        .save("Files/silver/dim_date.delta")

In [ ]:
print("✅ dim_date regenerated and saved to Silver.")

In [ ]:
spark.read.format("delta").load("Files/silver/dim_date.delta").filter("date >= '2023-01-01'").show(5)

In [ ]:
{"execution_finish_time":"2025-07-14T13:21:02.4319203Z","execution_start_time":"2025-07-14T13:20:53.4336847Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"0dab96bd-4d77-4b22-aa37-82fb8280bdd6","queued_time":"2025-07-14T13:20:41.8857747Z","session_id":"c3a2b1b9-6635-4443-945a-3e7b00f98c6a","session_start_time":"2025-07-14T13:20:41.8868106Z","spark_pool":null,"state":"finished","statement_id":3,"statement_ids":[3]}

In [ ]:
✅ dim_date regenerated and saved to Silver.
    +----------+--------+---+-----+-------+----+-------+----------+----------+---------+
    |      date| sk_date|day|month|quarter|year|weekday|week_of_yr|month_name| day_name|
    +----------+--------+---+-----+-------+----+-------+----------+----------+---------+
    |2023-01-01|20230101|  1|    1|      1|2023|      1|        52|   January|   Sunday|
    |2023-01-02|20230102|  2|    1|      1|2023|      2|         1|   January|   Monday|
    |2023-01-03|20230103|  3|    1|      1|2023|      3|         1|   January|  Tuesday|
    |2023-01-04|20230104|  4|    1|      1|2023|      4|         1|   January|Wednesday|
    |2023-01-05|20230105|  5|    1|      1|2023|      5|         1|   January| Thursday|
    +----------+--------+---+-----+-------+----+-------+----------+----------+---------+
    only showing top 5 rows

In [ ]:
fact_serviceinterruptions

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import sha2, col, concat_ws, upper, trim

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# =============================
    # 1. Load Silver Cleaned Dataset
    # =============================
    print("📥 Loading Silver cleaned dataset...")
    df = spark.read.format("delta").load("Files/silver/12_continuidade_de_servico_indicadores_gerais_de_continuidade_de_servico_cleaned")
    print(f"✅ Rows loaded from Silver: {df.count()}")
    df.printSchema()

In [ ]:
# =============================
    # 2. Normalize for SK generation
    # =============================
    print("🔧 Normalizing district and municipality for SK generation...")

In [ ]:
df = (
        df
        .withColumn("district", upper(trim(col("district"))))
        .withColumn("municipality", upper(trim(col("municipality"))))
        .withColumn("sk_municipality", sha2(concat_ws("-", col("district"), col("municipality")), 256))
    )

In [ ]:
# =============================
    # 3. Generate sk_date from year
    # =============================
    print("📆 Generating surrogate date key (sk_date)...")

In [ ]:
df = df.withColumn("sk_date", col("ano").cast("int") * 10000 + 101)  # Format: YYYY0101

In [ ]:
# =============================
    # 4. Select final fact columns
    # =============================
    print("🧱 Preparing fact table columns...")

In [ ]:
fact_df = df.select(
        "sk_date", "sk_municipality", "codigo_concelho", "ano", "zona_rqs",
        col("saifi_at_num").cast("double"),
        col("saidi_at_min").cast("double"),
        col("maifi_at_num").cast("double"),
        col("tiepi_mt_min").cast("double"),
        col("end_mt_mwh").cast("double"),
        col("saifi_mt_num").cast("double"),
        col("saidi_mt_min").cast("double"),
        col("maifi_mt_num").cast("double"),
        col("saifi_bt_num").cast("double"),
        col("saidi_bt_min").cast("double")
    )

In [ ]:
# =============================
    # 5. Preview and Save to Gold
    # =============================
    print("🔍 Preview of fact_serviceinterruptions:")
    fact_df.show(10, truncate=False)
    fact_df.printSchema()

In [ ]:
table_name = "fact_serviceinterruptions"
    gold_path = f"Files/gold/{table_name}.delta"

In [ ]:
print("💾 Saving to Gold layer...")
    fact_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(gold_path)
    print(f"✅ Saved to Gold path: {gold_path}")

In [ ]:
print("🗃️ Registering as SQL table...")
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    fact_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable(table_name)
    print(f"✅ SQL Table '{table_name}' registered and ready for Power BI.")

In [ ]:
{"execution_finish_time":"2025-07-14T13:25:15.296816Z","execution_start_time":"2025-07-14T13:24:28.9602894Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"413ef5da-9560-4578-af6a-5884d3efb2c4","queued_time":"2025-07-14T13:24:28.9592107Z","session_id":"c3a2b1b9-6635-4443-945a-3e7b00f98c6a","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":4,"statement_ids":[4]}

In [ ]:
📥 Loading Silver cleaned dataset...
    ✅ Rows loaded from Silver: 11120
    root
     |-- ano: string (nullable = true)
     |-- nuts_iii: string (nullable = true)
     |-- codigo_concelho: string (nullable = true)
     |-- concelho: string (nullable = true)
     |-- zona_rqs: string (nullable = true)
     |-- saifi_at_num: string (nullable = true)
     |-- saidi_at_min: string (nullable = true)
     |-- maifi_at_num: string (nullable = true)
     |-- tiepi_mt_min: string (nullable = true)
     |-- end_mt_mwh: string (nullable = true)
     |-- saifi_mt_num: string (nullable = true)
     |-- saidi_mt_min: string (nullable = true)
     |-- maifi_mt_num: string (nullable = true)
     |-- saifi_bt_num: string (nullable = true)
     |-- saidi_bt_min: string (nullable = true)
     |-- district: string (nullable = true)
     |-- municipality: string (nullable = true)
     |-- sk_municipality: string (nullable = true)

In [ ]:
🔧 Normalizing district and municipality for SK generation...
    📆 Generating surrogate date key (sk_date)...
    🧱 Preparing fact table columns...
    🔍 Preview of fact_serviceinterruptions:
    +--------+----------------------------------------------------------------+---------------+----+--------+------------+------------+------------+----------------+----------------+-----------------+----------------+----------------+-----------------+----------------+
    |sk_date |sk_municipality                                                 |codigo_concelho|ano |zona_rqs|saifi_at_num|saidi_at_min|maifi_at_num|tiepi_mt_min    |end_mt_mwh      |saifi_mt_num     |saidi_mt_min    |maifi_mt_num    |saifi_bt_num     |saidi_bt_min    |
    +--------+----------------------------------------------------------------+---------------+----+--------+------------+------------+------------+----------------+----------------+-----------------+----------------+----------------+-----------------+----------------+
    |20220101|68081357adc2636e3fdf27341130e238619f9971b9fa92a2954d24fc6e5f71d0|801            |2022|Zona A  |0.0         |0.0         |0.0         |35.0886618864081|9.80358647705634|0.739018584966534|33.7200185241627|2.10893914262048|0.731583091228553|35.9844072128545|
    |20220101|9d93c97b4d68506ed8448b7c6939fea257c9fdc505d9418a1a8af66d6bf030d3|1509           |2022|Concelho|0.0         |0.0         |1.0         |55.4735827389563|13.0337863884829|3.14141541612765 |53.4616789451799|15.1497877581598|3.77815531299883 |81.586029824039 |
    |20220101|153c8701e194af333c983d4f6da2e9fb8e7027b76ddc3a2f6563bb6adee359cf|802            |2022|Zona A  |0.0         |0.0         |0.0         |0.0             |0.0             |0.0              |0.0             |0.0             |0.0              |0.0             |
    |20220101|a04818d85609e86dd1d8cd0e28ad8d0ae694406fe10b311872f51591cc298c3b|811            |2022|Zona C  |0.0         |0.0         |0.0         |90.252065339809 |29.0793308935975|3.03753477448863 |94.7374221452774|9.49810118117486|2.83175710818914 |110.804005348286|
    |20220101|7236be143225509fcf9f73ebd51813ae7ad7cecb0cb23038c4297096c26e546a|813            |2022|Zona A  |0.0         |0.0         |0.0         |0.0             |0.0             |0.0              |0.0             |0.0             |0.0              |0.0             |
    |20220101|afe3601fd44acfa81074a3abdb4ba6bb6fa9b15f1f3ba0f845596411fa498a49|807            |2022|Zona C  |0.0         |0.0         |0.0         |192.316027397741|34.0141928013306|3.4969838877422  |193.970817753884|10.785846813549 |4.20326324037857 |209.214017536096|
    |20220101|aa32869c2f57a4132f2138c6c0d339f5cbeb8dcc0db380a616754544be7bf531|808            |2022|Zona C  |0.0         |0.0         |1.0         |45.5573154001483|21.5123580234781|1.64314475381734 |58.8978665378517|5.8200660355772 |1.78464924253625 |74.6284481954999|
    |20220101|f8db8f0291bca6ace3c9e67e0ae17f0823317edfc6363b7a5baa5ef3f6222b4b|816            |2022|Zona B  |0.0         |0.0         |0.0         |21.936907766801 |2.54752669647096|1.13998010490142 |21.2619319158075|6.65947504069451|1.26064176464759 |33.7047540223093|
    |20220101|050f6c0aaec4f008680f565c5b636c8ce82e1cdbb584e6a8dddb708fd17388bf|815            |2022|Zona C  |0.0         |0.0         |0.0         |155.899773218814|14.0969549164253|3.125            |191.935069444444|8.32172688422688|3.87374588004142 |222.378466486348|
    |20220101|050f6c0aaec4f008680f565c5b636c8ce82e1cdbb584e6a8dddb708fd17388bf|815            |2022|Zona A  |0.0         |0.0         |0.0         |0.0             |0.0             |0.0              |0.0             |0.0             |0.0              |0.0             |
    +--------+----------------------------------------------------------------+---------------+----+--------+------------+------------+------------+----------------+----------------+-----------------+----------------+----------------+-----------------+----------------+
    only showing top 10 rows

In [ ]:
root
     |-- sk_date: integer (nullable = true)
     |-- sk_municipality: string (nullable = true)
     |-- codigo_concelho: string (nullable = true)
     |-- ano: string (nullable = true)
     |-- zona_rqs: string (nullable = true)
     |-- saifi_at_num: double (nullable = true)
     |-- saidi_at_min: double (nullable = true)
     |-- maifi_at_num: double (nullable = true)
     |-- tiepi_mt_min: double (nullable = true)
     |-- end_mt_mwh: double (nullable = true)
     |-- saifi_mt_num: double (nullable = true)
     |-- saidi_mt_min: double (nullable = true)
     |-- maifi_mt_num: double (nullable = true)
     |-- saifi_bt_num: double (nullable = true)
     |-- saidi_bt_min: double (nullable = true)

In [ ]:
💾 Saving to Gold layer...
    ✅ Saved to Gold path: Files/gold/fact_serviceinterruptions.delta
    🗃️ Registering as SQL table...
    ✅ SQL Table 'fact_serviceinterruptions' registered and ready for Power BI.

In [ ]:
fact_energyconsumption_hourly

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, to_date

In [ ]:
# Start Spark session
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Cleaned Silver Tables
    # ============================
    df_cp = spark.read.format("delta").load("Files/silver/consumos_horario_codigo_postal_cleaned.delta")
    dim_date = spark.read.format("delta").load("Files/silver/dim_date.delta")
    dim_zip = spark.read.format("delta").load("Files/silver/dim_zipcode.delta")

In [ ]:
print("✅ Cleaned tables loaded")

In [ ]:
# ============================
    # 2. Convert datahora to Date
    # ============================
    df_cp = df_cp.withColumn("date", to_date(col("datahora")))
    print("📅 Converted 'datahora' to Date format")

In [ ]:
# ============================
    # 3. Join with dim_date
    # ============================
    df_cp = df_cp.join(
        dim_date.select(col("date").alias("dim_date"), "sk_date"),
        df_cp["date"] == col("dim_date"),
        how="left"
    )
    print("🔗 Joined with dim_date")

In [ ]:
# ============================
    # 4. Join with dim_zipcode
    # ============================
    # Remove any existing duplicate to avoid ambiguity
    if "sk_zipcode" in df_cp.columns:
        df_cp = df_cp.drop("sk_zipcode")

In [ ]:
df_cp = df_cp.join(
        dim_zip.select(col("zipcode"), col("sk_zipcode")),
        df_cp["codigo_postal"] == col("zipcode"),
        how="left"
    )
    print("🔗 Joined with dim_zipcode")

In [ ]:
# ============================
    # 5. Select Final Columns
    # ============================
    df_fact = df_cp.select(
        "sk_date",
        "sk_zipcode",
        "dt_consumo",
        "hr_consumo",
        "consumo",
        "dia_semana"
    )

In [ ]:
df_fact.show(10, truncate=False)

In [ ]:
# ============================
    # 6. Save to Gold Layer and SQL Table
    # ============================
    df_fact.write.mode("overwrite").format("delta").save("Files/gold/fact_energyconsumption_horaria.delta")
    print("💾 Saved to Gold layer as Delta")

In [ ]:
df_fact.write.mode("overwrite").format("delta").saveAsTable("fact_energyconsumption_hourly")
    print("✅ fact_energyconsumption_hourly available as SQL table")

In [ ]:
{"execution_finish_time":"2025-06-16T23:15:30.6397Z","execution_start_time":"2025-06-16T23:15:16.2694573Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"3f04db0b-62fc-4b1f-84c8-63621020b40c","queued_time":"2025-06-16T23:15:16.268275Z","session_id":"62e18c5d-4bd9-4dca-a351-fa5a081ca38c","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":18,"statement_ids":[18]}

In [ ]:
✅ Cleaned tables loaded
    📅 Converted 'datahora' to Date format
    🔗 Joined with DimDate
    🔗 Joined with DimZipCode
    +--------+----------------------------------------------------------------+----------+----------+------------------+----------+
    |sk_date |sk_zipcode                                                      |dt_consumo|hr_consumo|consumo           |dia_semana|
    +--------+----------------------------------------------------------------+----------+----------+------------------+----------+
    |20230319|44864c96fa1c36602f0d045b268981b6cab638a60fc207c89e3bba235f619450|2023-03-19|18:00     |6282.611569035354 |Domingo   |
    |20230201|a025f0314b164d72f92e11bd860ebbc0558addbeb5cf3ae1f06c4eded1d63ddc|2023-02-01|19:00     |8067.177903197795 |Quarta    |
    |20230202|6c6ece85b5d6ea8c5b1901b34909d170569478006b74472e49d7535ab4b4a94d|2023-02-02|22:00     |15586.610308574363|Quinta    |
    |20230222|8ace90d8ffdf2ee4bcd149c56579f93598080a47502ab616bc7c1c36b37e199f|2023-02-22|08:00     |7482.595338508814 |Quarta    |
    |20230203|b3e34e9fe5a79d4e8753d0ad4107d0af969d8faaefb88fbd68316950fa2a9242|2023-02-03|20:00     |17077.941720628023|Sexta     |
    |20230225|541eead0d9aecd91a4e591be604dbe311bdebdbf25c85821fabf878018f0504c|2023-02-25|23:00     |16547.202909233984|Sábado    |
    |20221222|f338a66d1dc630488f938baf32f8bd95bf6e26fb48e022258e5349c431b618c1|2022-12-22|07:00     |3187.701516925412 |Quinta    |
    |20230921|1625f9db144171f78dc64afa00f5f5065f176d05eb8f9dcb11f6c8cd3624aaa6|2023-09-21|20:00     |12066.674853105147|Quinta    |
    |20230907|47c04d7995ba767c5024eee3ade65a7aab34ac951c17a3e4265a4a5602e18592|2023-09-07|02:00     |6668.482964061213 |Quinta    |
    |20230902|1625f9db144171f78dc64afa00f5f5065f176d05eb8f9dcb11f6c8cd3624aaa6|2023-09-02|04:00     |6095.369720809502 |Sábado    |
    +--------+----------------------------------------------------------------+----------+----------+------------------+----------+
    only showing top 10 rows

In [ ]:
💾 Saved to Gold layer as Delta
    ✅ Fact_EnergyConsumption_Hourly available as SQL table

In [ ]:
fact_energyconsumption_minicipality

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, sha2, concat_ws, to_date, expr, upper, trim, last_day, lit
    from pyspark.sql.functions import sum as _sum

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Cleaned and Dimension Tables
    # ============================
    df_cleaned = spark.read.format("delta").load("Files/silver/3_consumos_faturados_por_municipio-ultimos_10_anos_cleaned.delta")
    df_date = spark.read.format("delta").load("Files/silver/dim_date.delta")
    df_muni = spark.read.format("delta").load("Files/silver/dim_municipality.delta")

In [ ]:
print("✅ Cleaned and dimension tables loaded")

In [ ]:
# ============================
    # 2. Drop ambiguous SKs if needed
    # ============================
    if "sk_municipality" in df_cleaned.columns:
        df_cleaned = df_cleaned.drop("sk_municipality")
        print("🧹 Dropped existing sk_municipality to avoid ambiguity")

In [ ]:
# ============================
    # 3. Convert 'data' (yyyy-MM) → último dia do mês
    # ============================
    df_cleaned = df_cleaned.withColumn(
        "date", last_day(to_date(concat_ws("-", col("data").cast("string"), lit("01"))))
    )
    print("📅 Converted 'data' to last day of month")

In [ ]:
# ============================
    # 4. Join with dim_date
    # ============================
    df_cleaned = df_cleaned.join(
        df_date.select(col("date").alias("dim_date"), "sk_date"),
        df_cleaned["date"] == col("dim_date"),
        how="left"
    )
    print("🔗 Joined with dim_date")

In [ ]:
# ============================
    # 5. Join with dim_municipality
    # ============================
    df_cleaned = df_cleaned.join(
        df_muni.select("sk_municipality", "district", "municipality"),
        on=["district", "municipality"],
        how="left"
    )
    print("🔗 Joined with dim_municipality")

In [ ]:
# ============================
    # 6. Cast energia to double
    # ============================
    df_cleaned = df_cleaned.withColumn("energia_ativa_kwh", col("energia_ativa_kwh").cast("double"))
    print("⚙️ Converted energia_ativa_kwh to double")

In [ ]:
# ============================
    # 6.1 Filter out "OUTROS <DISTRITO>" rows
    # ============================
    df_cleaned = df_cleaned.filter("sk_municipality IS NOT NULL")
    print("🧼 Filtered out rows with NULL sk_municipality (e.g. 'OUTROS')")

In [ ]:
# ============================
    # 7. Aggregate by sk_date and sk_municipality
    # ============================
    df_fact = df_cleaned.groupBy("sk_date", "sk_municipality").agg(
        _sum("energia_ativa_kwh").alias("energia_ativa_kwh")
    )
    print("📊 Aggregated consumption by sk_date and sk_municipality")

In [ ]:
# ============================
    # 8. Show sample rows
    # ============================
    df_fact.show(10, truncate=False)

In [ ]:
# ============================
    # 9. Save to Gold and SQL Table
    # ============================
    df_fact.write.mode("overwrite").format("delta").save("Files/gold/fact_energyconsumption_municipio.delta")
    print("💾 Saved to Gold layer as Delta")

In [ ]:
df_fact.write.mode("overwrite").format("delta").saveAsTable("fact_energyconsumption_municipality")
    print("✅ Fact_EnergyConsumption_Municipality available as SQL table")

In [ ]:
{"execution_finish_time":"2025-06-16T23:41:39.721698Z","execution_start_time":"2025-06-16T23:41:27.5744244Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"b9063a93-3be5-4098-9110-02a91adbe8ad","queued_time":"2025-06-16T23:41:27.573229Z","session_id":"62e18c5d-4bd9-4dca-a351-fa5a081ca38c","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":34,"statement_ids":[34]}

In [ ]:
✅ Cleaned and dimension tables loaded
    🧹 Dropped existing sk_municipality to avoid ambiguity
    📅 Converted 'data' to last day of month
    🔗 Joined with DimDate
    🔗 Joined with DimMunicipality
    ⚙️ Converted energia_ativa_kwh to double
    🧼 Filtered out rows with NULL sk_municipality (e.g. 'OUTROS')
    📊 Aggregated consumption by sk_date and sk_municipality
    +--------+----------------------------------------------------------------+------------------+
    |sk_date |sk_municipality                                                 |energia_ativa_kwh |
    +--------+----------------------------------------------------------------+------------------+
    |20240131|06600545128a8bbb01061378131699d37862288096b71fd0e8377d976ecec2ed|4944356.75        |
    |20240131|f66048b6fc1045622c41beadc3c858869065b30aca36665114de9dd93a932280|1194109.659       |
    |20240229|3dc1620ef3ad86c1934f4cd67c43582325ee3746745ef814ac16d60a1434fb7f|574484.786        |
    |20231130|df5583e09f57f5b32c4ecb311c686d48d12761ea7bab100caea7b9b81a042109|899644.7540000001 |
    |20231231|9621496898bf8bc7cd67d440d56c5e632bc29a0bcce8edd198f4459b17cdedfd|1405899.182       |
    |20231231|a79128db844c02483d44e128263c81b0f3067a6ed14246eed583d345bdfdb406|1.1499708989E7    |
    |20240131|4664effdfd10c006c69d0d80d7e9218e241cef39fdc2a66e561f34908290c411|5278992.177999999 |
    |20230228|ef0692a3fac4ba17f47c1117d319250c838d05c9896f315cf23b67dc8124d13f|1012567.3899999999|
    |20230228|849fc1882a0add2cc6f87d9bc4c5fb03ae19f137cf96da61f4cd305e84d86feb|3569264.1029999997|
    |20230331|d94ddc3dc7a9a2e97841c518361778856c6935212ad3112c1dde08c40104c0b7|4291827.392999999 |
    +--------+----------------------------------------------------------------+------------------+
    only showing top 10 rows

In [ ]:
💾 Saved to Gold layer as Delta
    ✅ Fact_EnergyConsumption_Municipality available as SQL table

In [ ]:
fact_energyconsumption_zipcode

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, sha2, concat_ws, to_date, last_day, lit
    from pyspark.sql.functions import sum as _sum

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Cleaned and Dimension Tables
    # ============================
    df_cleaned = spark.read.format("delta").load("Files/silver/02_consumos_faturados_por_codigo_postal_ultimos_5_anos_cleaned.delta")
    df_date = spark.read.format("delta").load("Files/silver/dim_date.delta")
    df_zip = spark.read.format("delta").load("Files/silver/dim_zipcode.delta")

In [ ]:
print("✅ Cleaned and dimension tables loaded")

In [ ]:
# ============================
    # 2. Drop ambiguous SKs if needed
    # ============================
    if "sk_zipcode" in df_cleaned.columns:
        df_cleaned = df_cleaned.drop("sk_zipcode")
        print("🧹 Dropped existing sk_zipcode to avoid ambiguity")

In [ ]:
# ============================
    # 3. Convert 'date' (yyyy-MM) → último dia do mês
    # ============================
    df_cleaned = df_cleaned.withColumn(
        "date", last_day(to_date(concat_ws("-", col("date").cast("string"), lit("01"))))
    )
    print("📅 Converted 'date' to last day of month")

In [ ]:
# ============================
    # 4. Join with dim_date
    # ============================
    df_cleaned = df_cleaned.join(
        df_date.select(col("date").alias("dim_date"), "sk_date"),
        df_cleaned["date"] == col("dim_date"),
        how="left"
    )
    print("🔗 Joined with dim_date")

In [ ]:
# ============================
    # 5. Join with dim_zipcode
    # ============================
    df_cleaned = df_cleaned.join(
        df_zip.select("sk_zipcode", "zipcode"),
        on="zipcode",
        how="left"
    )
    print("🔗 Joined with dim_zipcode")

In [ ]:
# ============================
    # 6. Cast energia to double
    # ============================
    df_cleaned = df_cleaned.withColumn("energia_ativa_kwh", col("energia_ativa_kwh").cast("double"))
    print("⚙️ Converted energia_ativa_kwh to double")

In [ ]:
# ============================
    # 7. Filter out NULLs (if any)
    # ============================
    df_cleaned = df_cleaned.filter("sk_date IS NOT NULL AND sk_zipcode IS NOT NULL")
    print("🧼 Filtered out rows with NULL sk_date or sk_zipcode")

In [ ]:
# ============================
    # 8. Aggregate
    # ============================
    df_fact = df_cleaned.groupBy("sk_date", "sk_zipcode").agg(
        _sum("energia_ativa_kwh").alias("energia_ativa_kwh")
    )
    print("📊 Aggregated consumption by sk_date and sk_zipcode")

In [ ]:
# ============================
    # 9. Show sample
    # ============================
    df_fact.show(10, truncate=False)

In [ ]:
# ============================
    # 10. Save to Gold
    # ============================
    df_fact.write.mode("overwrite").format("delta").save("Files/gold/fact_energyconsumption_zipcode.delta")
    print("💾 Saved to Gold layer as Delta")

In [ ]:
df_fact.write.mode("overwrite").format("delta").saveAsTable("fact_energyconsumption_zipcode")
    print("✅ fact_energyconsumption_zipcode available as SQL table")

In [ ]:
{"execution_finish_time":"2025-06-17T00:28:51.7832189Z","execution_start_time":"2025-06-17T00:28:29.0879167Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"0219ecc8-72e2-4e62-baea-c28d0d81b5f9","queued_time":"2025-06-17T00:28:29.0867472Z","session_id":"51dd5d37-4239-4b07-afa9-5612272f5a0a","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":4,"statement_ids":[4]}

In [ ]:
✅ Cleaned and dimension tables loaded
    🧹 Dropped existing sk_zipcode to avoid ambiguity
    📅 Converted 'date' to last day of month
    🔗 Joined with DimDate
    🔗 Joined with DimZipCode
    ⚙️ Converted energia_ativa_kwh to double
    🧼 Filtered out rows with NULL sk_date or sk_zipcode
    📊 Aggregated consumption by sk_date and sk_zipcode
    +--------+----------------------------------------------------------------+-----------------+
    |sk_date |sk_zipcode                                                      |energia_ativa_kwh|
    +--------+----------------------------------------------------------------+-----------------+
    |20240531|cead18006a4de84ec2152071abe3deaf2bb386a00070f29f69c6e534c3d386f0|868439.882       |
    |20241031|5616b00748424b555643e35b623f2e82430dc57e936b0da684c5e64b295f00b8|7.2211463208E7   |
    |20241130|0ec89b31a9f42decb7bad3c678c0994266ae6382435ec44c7ed3144b0f3dae2a|9939904.535      |
    |20241130|41ac9f4b6edb4de8be1a0542d145603d7f2dc2aa8d878691d750860653647eeb|5624319.718      |
    |20250131|7b3087f7c84638235f3faf074627cde9c39a713dbe6a6c9984e84f2242df4fca|3959730.366      |
    |20250131|90c246a812e634906611787f035b1d4841b162d62aa6a4145b1b2e32e526d54e|2056531.85       |
    |20221031|327e892542e0f4097f90d914962a75ddbe9cb0577007d7b7d45dea310086bb97|11632.249        |
    |20230331|dea1ae8613ee2a5ef5504d98bd969bd46b4c75e9c898341f497c74684599030c|9357594.054      |
    |20230430|7263af08814e11782e313d81492e2c644c3152b0f42deeff9c0efa80667b5094|749567.829       |
    |20230531|620e9c1f98e4730c1968dd7e14627cdff6689e377fa8ff7d5be4fd3540b57543|4211069.218      |
    +--------+----------------------------------------------------------------+-----------------+
    only showing top 10 rows

In [ ]:
💾 Saved to Gold layer as Delta
    ✅ Fact_EnergyConsumption_ZipCode available as SQL table

In [ ]:
fact_gridcapacity

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, trim, upper

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# --------------------------
    # STEP 1: Load Cleaned Data & dim_municipality
    # --------------------------
    df_cleaned = spark.read.format("delta").load("Files/silver/carga_na_subestacao_cleaned.delta")
    dim_municipality = spark.read.format("delta").load("Files/silver/dim_municipality.delta")

In [ ]:
print("✅ Cleaned and dimension tables loaded")

In [ ]:
# --------------------------
    # STEP 2: Normalize 'municipality' for join
    # --------------------------
    df_cleaned = df_cleaned.withColumn("municipality", upper(trim(col("nome"))))
    dim_municipality = dim_municipality.withColumn("municipality", upper(trim(col("municipality"))))

In [ ]:
print("🧼 Normalized 'municipality' for both tables")

In [ ]:
# --------------------------
    # STEP 3: Join on 'municipality' and project required columns only
    # --------------------------
    df_joined = (
        df_cleaned.alias("fact")
        .join(
            dim_municipality.select("municipality", "sk_municipality").alias("dim"),
            on="municipality",
            how="inner"
        )
        .select(
            col("fact.ano").cast("int").alias("year"),
            col("dim.sk_municipality"),
            col("fact.potencia_instalada").cast("double"),
            col("fact.potencia_garantida").cast("double"),
            col("fact.potencia_nao_garantida").cast("double"),
            col("fact.disponibilidade").cast("double"),
            col("fact.carga_natural").cast("double")
        )
    )

In [ ]:
print("🔗 Joined with dim_municipality on 'municipality'")
    print("⚙️ Selected and casted relevant columns")

In [ ]:
# --------------------------
    # STEP 4: Preview result
    # --------------------------
    df_joined.show(10, truncate=False)

In [ ]:
# --------------------------
    # STEP 5: Save to Gold layer
    # --------------------------
    df_joined.write.mode("overwrite").format("delta").save("Files/gold/fact_gridcapacity.delta")

In [ ]:
print("💾 Saved to Gold layer as Delta")
    print("✅ fact_gridcapacity available as SQL table")

In [ ]:
{"execution_finish_time":"2025-06-23T12:31:56.1455814Z","execution_start_time":"2025-06-23T12:31:40.9423121Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"698816d5-6117-4c13-8024-d9ea5d09f223","queued_time":"2025-06-23T12:31:27.4853332Z","session_id":"052e33ea-c3bb-4caa-aaa2-7d12adc733ae","session_start_time":"2025-06-23T12:31:27.4863849Z","spark_pool":null,"state":"finished","statement_id":3,"statement_ids":[3]}

In [ ]:
✅ Cleaned and dimension tables loaded
    🧼 Normalized 'municipality' for both tables
    🔗 Joined with DimMunicipality on 'municipality'
    ⚙️ Selected and casted relevant columns
    +----+----------------------------------------------------------------+------------------+------------------+----------------------+------------------+-------------+
    |year|sk_municipality                                                 |potencia_instalada|potencia_garantida|potencia_nao_garantida|disponibilidade   |carga_natural|
    +----+----------------------------------------------------------------+------------------+------------------+----------------------+------------------+-------------+
    |2024|c993313f3f271be73c32fef0d16b407c5db50f996287f460f76245e6a38392d2|63.0              |57.8              |5.200000000000003     |25.578            |32.222       |
    |2024|24e9ceba453a0004de633b3adcd7a6c845dc4b243243f8d60c7493ca868b1641|20.0              |10.0              |10.0                  |0.0               |12.38        |
    |2024|3b8e3b30f3a250edd309725c9c4a435c23f26c537550e12a2f6c765c3a3e30df|40.0              |34.0              |6.0                   |4.4830000000000005|29.517       |
    |2024|c022c4305a1cd5ae9eb424345f4fa01c4d6f5959e87892eaaf1324cbabb021fd|5.0               |2.5               |2.5                   |0.0               |2.938        |
    |2024|cf3e9b4d86caddfd970f15eebcd0340e4b2d181e7a0b7c9b32235ba2b846aedb|20.0              |10.0              |10.0                  |2.333             |7.667        |
    |2024|fcb15ff47cd962e2519b117d25b2f62a6d44b995ac77812e86980e53ec2c0f56|20.0              |10.0              |10.0                  |0.0               |10.082       |
    |2024|fcb15ff47cd962e2519b117d25b2f62a6d44b995ac77812e86980e53ec2c0f56|20.0              |10.0              |10.0                  |4.764             |5.236        |
    |2024|8777f2fd9d0308ec352295a83e91921396dbd46778730ed87c241c1e98d765d3|63.0              |48.825            |14.174999999999995    |16.919000000000004|31.906       |
    |2024|fdc6ce3cd22852873d4e9695772d68785b8d30c8c1f48fbcc88ef605833cf61a|20.0              |16.0              |4.0                   |3.4030000000000005|12.597       |
    |2024|fdc6ce3cd22852873d4e9695772d68785b8d30c8c1f48fbcc88ef605833cf61a|20.0              |16.0              |4.0                   |0.0               |16.099       |
    +----+----------------------------------------------------------------+------------------+------------------+----------------------+------------------+-------------+
    only showing top 10 rows

In [ ]:
💾 Saved to Gold layer as Delta
    ✅ Fact_GridCapacity available as SQL table

In [ ]:
fact_renewableadoption

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, lit, to_date, date_format, sha2, concat_ws, when
    from pyspark.sql import DataFrame
    from functools import reduce

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Silver Cleaned Datasets
    # ============================
    print("📥 Loading Silver cleaned datasets...")
    df_plr = spark.read.format("delta").load("Files/silver/25_plr_producao_renovavel_cleaned.delta")
    df_centrais = spark.read.format("delta").load("Files/silver/26_centrais_cleaned.delta")
    df_injection = spark.read.format("delta").load("Files/silver/energia_injetada_na_rede_de_distribuicao_cleaned.delta")
    print("✅ Silver datasets loaded")

In [ ]:
# ============================
    # 2. Normalize Dates and sk_date
    # ============================
    def add_sk_date_from_semester(df):
        df = df.withColumn(
            "date",
            when(col("semestre") == "1", concat_ws("-", col("ano"), lit("06"), lit("30")))
            .when(col("semestre") == "2", concat_ws("-", col("ano"), lit("12"), lit("31")))
        )
        return df.withColumn("date", to_date("date", "yyyy-MM-dd")) \
                 .withColumn("sk_date", date_format("date", "yyyyMMdd").cast("int"))

In [ ]:
df_plr = add_sk_date_from_semester(df_plr)
    df_centrais = add_sk_date_from_semester(df_centrais)

In [ ]:
# ============================
    # 3. Select & Rename Columns
    # ============================
    df_plr_sel = df_plr.select(
        "sk_date",
        "sk_municipality",
        lit("e182f788feafe6f93dac5016ff45d88df5ee9def3dff67a9357999715a736803").alias("sk_policy"),
        col("pedidos_de_ligacao_a_rede_executados").cast("int").alias("num_installations"),
        col("potencia_de_ligacao").cast("double").alias("total_power_kw"),
        lit(None).cast("double").alias("national_production_gwh"),
        lit(None).cast("string").alias("sk_technology")
    )

In [ ]:
df_centrais_sel = df_centrais.select(
        "sk_date",
        "sk_municipality",
        lit(None).cast("string").alias("sk_policy"),
        col("processos_concluidos").cast("int").alias("num_installations"),
        col("potencia_de_ligacao").cast("double").alias("total_power_kw"),
        lit(None).cast("double").alias("national_production_gwh"),
        lit(None).cast("string").alias("sk_technology")
    )

In [ ]:
# ============================
    # 4. Expand Renewable Injection by Technology
    # ============================
    df_injection = df_injection.withColumn("sk_date", date_format(col("date").cast("date"), "yyyyMMdd").cast("int"))

In [ ]:
df_wind = df_injection.select(
        "sk_date",
        lit(None).cast("string").alias("sk_municipality"),
        lit(None).cast("string").alias("sk_policy"),
        lit(None).cast("int").alias("num_installations"),
        lit(None).cast("double").alias("total_power_kw"),
        (col("wind_kwh") / 1e6).alias("national_production_gwh"),
        sha2(lit("Eólica"), 256).alias("sk_technology")
    )

In [ ]:
df_solar = df_injection.select(
        "sk_date",
        lit(None).cast("string").alias("sk_municipality"),
        lit(None).cast("string").alias("sk_policy"),
        lit(None).cast("int").alias("num_installations"),
        lit(None).cast("double").alias("total_power_kw"),
        (col("solar_kwh") / 1e6).alias("national_production_gwh"),
        sha2(lit("Solar Fotovoltaica"), 256).alias("sk_technology")
    )

In [ ]:
df_hydro = df_injection.select(
        "sk_date",
        lit(None).cast("string").alias("sk_municipality"),
        lit(None).cast("string").alias("sk_policy"),
        lit(None).cast("int").alias("num_installations"),
        lit(None).cast("double").alias("total_power_kw"),
        (col("hydro_kwh") / 1e6).alias("national_production_gwh"),
        sha2(lit("Hídrica"), 256).alias("sk_technology")
    )

In [ ]:
# ============================
    # 5. Final Union & Save
    # ============================
    df_final = df_wind.unionByName(df_solar).unionByName(df_hydro)

In [ ]:
print("✅ Loaded cleaned Silver datasets.")

In [ ]:
print("📊 Preview final de fact_renewableadoption (por tecnologia):")
    df_final.show(10, truncate=False)

In [ ]:
# ============================
    # 6. Save to Gold Layer
    # ============================
    path_gold = "Files/gold/fact_renewableadoption.delta"
    print(f"💾 Saving fact_renewableadoption to Gold at {path_gold}")
    df_final.write.mode("append").format("delta").save(path_gold)

In [ ]:
# ============================
    # 7. Register as SQL Table
    # ============================
    print("🗃️ Registering as SQL Table...")
    spark.sql("DROP TABLE IF EXISTS fact_renewableadoption")
    df_final.write.mode("overwrite").format("delta").saveAsTable("fact_renewableadoption")

In [ ]:
print("✅ SQL Table 'fact_renewableadoption' registada com sucesso no metastore Fabric ⚡")

In [ ]:
{"execution_finish_time":"2025-06-27T23:20:10.6789386Z","execution_start_time":"2025-06-27T23:20:02.7262892Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"48b06731-7d58-42ab-9d96-96c21ffd76ac","queued_time":"2025-06-27T23:20:02.7250978Z","session_id":"62f3a26e-05ae-4dde-b7d2-bf31a7a8f0f0","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":6,"statement_ids":[6]}

In [ ]:
📥 Loading Silver cleaned datasets...
    ✅ Silver datasets loaded
    ✅ Loaded cleaned Silver datasets.
    📊 Preview final de Fact_RenewableAdoption (por tecnologia):
    +--------+---------------+---------+-----------------+--------------+-----------------------+----------------------------------------------------------------+
    |sk_date |sk_municipality|sk_policy|num_installations|total_power_kw|national_production_gwh|sk_technology                                                   |
    +--------+---------------+---------+-----------------+--------------+-----------------------+----------------------------------------------------------------+
    |20240320|NULL           |NULL     |NULL             |NULL          |18.90736               |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20240726|NULL           |NULL     |NULL             |NULL          |40.9059485             |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20230209|NULL           |NULL     |NULL             |NULL          |59.339387              |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20240522|NULL           |NULL     |NULL             |NULL          |27.10476025            |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20250403|NULL           |NULL     |NULL             |NULL          |66.34277475            |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20250106|NULL           |NULL     |NULL             |NULL          |58.8671175             |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20240630|NULL           |NULL     |NULL             |NULL          |11.89018475            |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20240926|NULL           |NULL     |NULL             |NULL          |81.78380925            |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20230917|NULL           |NULL     |NULL             |NULL          |73.36303325            |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    |20250425|NULL           |NULL     |NULL             |NULL          |38.499208              |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|
    +--------+---------------+---------+-----------------+--------------+-----------------------+----------------------------------------------------------------+
    only showing top 10 rows

In [ ]:
💾 Saving Fact_RenewableAdoption to Gold at Files/gold/Fact_RenewableAdoption.delta
    🗃️ Registering as SQL Table...
    ✅ SQL Table 'Fact_RenewableAdoption' registada com sucesso no metastore Fabric ⚡

In [ ]:
dim_installationtype

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import sha2, col

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # STEP 1: Definir os tipos de instalação
    # ============================
    df_dim_installation = spark.createDataFrame([
        ("PLR_RenewableConnection", "Ligação de produção renovável à rede (PLR)"),
        ("Central", "Central elétrica existente"),
        ("NationalTotal", "Produção total nacional"),
        ("RenewableInjection", "Energia renovável injetada na rede de distribuição")
    ], ["installation_type", "description"])

In [ ]:
# ============================
    # STEP 2: Gerar surrogate key
    # ============================
    df_dim_installation = df_dim_installation.withColumn(
        "sk_installation_type", sha2(col("installation_type"), 256)
    ).select("sk_installation_type", "installation_type", "description")

In [ ]:
# ============================
    # STEP 3: Guardar e Registar como SQL Table
    # ============================
    table_name = "dim_installationtype"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
# Guardar como Delta
    df_dim_installation.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .save(path_gold)

In [ ]:
print(f"💾 Saved {table_name} to Gold at {path_gold}")

In [ ]:
# Registar como tabela SQL
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    df_dim_installation.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable(table_name)

In [ ]:
print(f"✅ SQL Table '{table_name}' registada com sucesso no Fabric")

In [ ]:
# ============================
    # STEP 4: Preview
    # ============================
    print("📑 Schema da tabela dim_installationtype:")
    df_dim_installation.printSchema()

In [ ]:
print("🔍 Preview da dimensão:")
    df_dim_installation.show(truncate=False)

In [ ]:
{"execution_finish_time":"2025-06-27T21:40:06.9901685Z","execution_start_time":"2025-06-27T21:39:41.8624192Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"630ec00f-93f2-4dc1-a96d-6c60f4d248b3","queued_time":"2025-06-27T21:39:41.8610708Z","session_id":"1d5d1861-c930-4e10-8f88-13fdeacc0d7e","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":7,"statement_ids":[7]}

In [ ]:
💾 Saved Dim_InstallationType to Gold at Files/gold/Dim_InstallationType.delta
    ✅ SQL Table 'Dim_InstallationType' registada com sucesso no Fabric
    📑 Schema da tabela Dim_InstallationType:
    root
     |-- sk_installation_type: string (nullable = true)
     |-- installation_type: string (nullable = true)
     |-- description: string (nullable = true)

In [ ]:
🔍 Preview da dimensão:
    +----------------------------------------------------------------+-----------------------+--------------------------------------------------+
    |sk_installation_type                                            |installation_type      |description                                       |
    +----------------------------------------------------------------+-----------------------+--------------------------------------------------+
    |cd812bd96766b63f054062a53f1d66fd82c06f800ca614ab1264e9e2d80f50c7|PLR_RenewableConnection|Ligação de produção renovável à rede (PLR)        |
    |0200f0d078fcfa701ae188a25bfb1809c9488d72ad1bfdeba59b85c6935690fa|Central                |Central elétrica existente                        |
    |db467ee9772b1be4a61a986d036176888251eb523b860c85a770fec36093074e|NationalTotal          |Produção total nacional                           |
    |4042252ffaec354c5883c68e32386e962c3e83d685b7e579953cb75052511f35|RenewableInjection     |Energia renovável injetada na rede de distribuição|
    +----------------------------------------------------------------+-----------------------+--------------------------------------------------+

In [ ]:
fact_gridoperations

In [ ]:
from pyspark.sql import SparkSession 
    from pyspark.sql.functions import (
        concat_ws, lit, lpad, to_date, last_day, col,
        sum, count, split, date_format, regexp_extract, when, length
    )

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Cleaned Silver Datasets
    # ============================
    df_reads_raw = spark.read.format("delta").load("Files/silver/23_leituras_recolhidas_remotamente_cleaned.delta")
    df_orders_raw = spark.read.format("delta").load("Files/silver/16_pedidos_concluidos_plrs_cleaned.delta")
    print("✅ Loaded cleaned Silver datasets.")

In [ ]:
# ============================
    # 2. Clean df_reads
    # ============================
    df_reads = df_reads_raw \
        .filter(col("data").isNotNull()) \
        .withColumn(
            "date",
            when(length(col("data")) == 10, to_date(col("data"), "yyyy-MM-dd"))
            .when(length(col("data")) == 7, last_day(to_date(col("data"), "yyyy-MM")))
        ) \
        .filter(col("date").isNotNull()) \
        .withColumn("sk_date", date_format("date", "yyyyMMdd").cast("int")) \
        .filter((col("sk_date").isNotNull()) & (col("sk_date") > 19000000))

In [ ]:
# ============================
    # 3. Clean df_orders (já tem date e sk_date)
    # ============================
    df_orders = df_orders_raw \
        .filter(col("sk_date").isNotNull()) \
        .filter(col("sk_municipality").isNotNull()) \
        .select("sk_date", "sk_municipality", "pedidos")  # já está limpo

In [ ]:
# ============================
    # 4. Aggregations
    # ============================
    reads_agg = df_reads.groupBy("sk_date", "sk_municipality").agg(
        sum(col("cpes_com_leituras").cast("int")).alias("RemoteReads_Count")
    )

In [ ]:
orders_agg = df_orders.groupBy("sk_date", "sk_municipality").agg(
        sum("pedidos").alias("ServiceOrders_Count")
    )

In [ ]:
print("✅ Aggregations concluídas com sucesso.")

In [ ]:
# ============================
    # 5. Join Aggregates
    # ============================
    df_fact = reads_agg.join(orders_agg, on=["sk_date", "sk_municipality"], how="outer").na.fill(0)
    print("✅ Preview da Fact Table combinada:")
    df_fact.show(10, truncate=False)

In [ ]:
# ============================
    # 6. Save and Register SQL Table
    # ============================
    table_name = "fact_gridoperations"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
# Guardar como Delta na Gold Layer
    df_fact.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .save(path_gold)
    print(f"💾 Saved {table_name} to Gold at {path_gold}")

In [ ]:
# Registar como SQL Table visível no Power BI
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    df_fact.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable(table_name)
    print(f"✅ SQL Table '{table_name}' registada com sucesso no metastore Fabric 🎯")

In [ ]:
{"execution_finish_time":"2025-06-28T18:27:04.567095Z","execution_start_time":"2025-06-28T18:26:52.6623609Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"090e6207-e9ca-48fd-bafc-e41200529789","queued_time":"2025-06-28T18:26:52.6610708Z","session_id":"8afc2853-d9fe-4fe7-b578-be998f39163a","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":7,"statement_ids":[7]}

In [ ]:
✅ Loaded cleaned Silver datasets.
    ✅ Aggregations concluídas com sucesso.
    ✅ Preview da Fact Table combinada:
    +--------+----------------------------------------------------------------+-----------------+-------------------+
    |sk_date |sk_municipality                                                 |RemoteReads_Count|ServiceOrders_Count|
    +--------+----------------------------------------------------------------+-----------------+-------------------+
    |20210630|009f8ca79b63c5ed54e3c65b39302529954949020538da5209f6c993ee3e3385|0                |173                |
    |20210630|00ed80ce99753ed4d66f64d1ff3a9faad0fb2f23f5a2f934a98f2923083b9995|0                |222                |
    |20210630|0164dacb1b33e8ecedd9262db9316baece6b500d6ed6aa9707282a65b3b4db8c|0                |53                 |
    |20210630|01f8f6eb536c2ec47db6bf08999db978076d19ad04d234e7be4886d6fa52531d|0                |47                 |
    |20210630|032fa22e7767c6ef9718b55865ddce365f7537415568a2ca45f7b0ee67710d88|0                |213                |
    |20210630|03449b54a14c4ab8c119fe03278c208dc98f861fa43b76151c0c2a17f4fa7cc1|0                |71                 |
    |20210630|04051e189e0d70b955ffd7ed5831954e882b8f11d6cc8fb0e773ef28191dc2fd|0                |34                 |
    |20210630|042c9b16bf8dae32d4185a651112f6d7852e254fcfe38f6a91b62a828bfd45bd|0                |14                 |
    |20210630|04588e08c0ec8e61a0216e391351760cf60dae18d39e1db14ba3e9a8d220a323|0                |43                 |
    |20210630|05a0c565eb5f30c69360c1e74ddab325e04057f38a9c71457158d0a8dd2703ec|0                |81                 |
    +--------+----------------------------------------------------------------+-----------------+-------------------+
    only showing top 10 rows

In [ ]:
💾 Saved Fact_GridOperations to Gold at Files/gold/Fact_GridOperations.delta
    ✅ SQL Table 'Fact_GridOperations' registada com sucesso no metastore Fabric 🎯

In [ ]:
fact_serviceinterruptions

In [ ]:
from pyspark.sql.functions import sha2, col, to_timestamp, to_date, concat_ws, lit, lpad

In [ ]:
print("🧼 Cleaning and transforming outages_per_geography...")
    df_cleaned = df_outages \
        .withColumn("date", to_date(to_timestamp("extractiondatetime"))) \
        .withColumn("zipcode4", col("zipcode").substr(1, 4)) \
        .withColumn("sk_date", sha2(col("date").cast("string"), 256)) \
        .withColumn("sk_zipcode", sha2(col("zipcode4"), 256)) \
        .withColumn("sk_municipality", sha2(concat_ws("-", lit("Lisboa"), col("municipality")), 256)) \
        .withColumn("interruption_flag", col("interrupcao_ativa").cast("int"))

In [ ]:
print("📊 Selecting final fields for fact_serviceinterruptions...")
    fact_serviceinterruptions = df_cleaned.select(
        "sk_date",
        "sk_zipcode",
        "sk_municipality",
        "interruption_flag"
    )

In [ ]:
# Guardar como tabela Delta Gold
    fact_serviceinterruptions.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save("Files/gold/fact_serviceinterruptions.delta")

In [ ]:
# Registar como tabela SQL
    spark.sql("DROP TABLE IF EXISTS fact_serviceinterruptions")
    spark.sql("""
        CREATE TABLE fact_serviceinterruptions
        USING DELTA
        LOCATION 'Files/gold/fact_serviceinterruptions.delta'
    """)

In [ ]:
print("✅ fact_serviceinterruptions criada com sucesso.")

In [ ]:
{"execution_finish_time":"2025-06-18T20:56:24.7737157Z","execution_start_time":"2025-06-18T20:55:28.0815251Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"c2307957-79d6-4a3d-993b-c95dd8c13b94","queued_time":"2025-06-18T20:55:28.0801451Z","session_id":"886920ea-a949-4033-81d6-cca974ac4db8","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":8,"statement_ids":[8]}

In [ ]:
🧼 Cleaning and transforming outages_per_geography...
    📊 Selecting final fields for Fact_ServiceInterruptions...
    ✅ Fact_ServiceInterruptions criada com sucesso.

In [ ]:
fact_electricmobility

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import (
        col, upper, trim, regexp_extract, sum as _sum, when,
        concat_ws, lit, to_date, date_format
    )

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Cleaned Silver Tables
    # ============================
    df_ves = spark.read.format("delta").load("Files/silver/postos_carregamento_ves_cleaned.delta")
    df_plr = spark.read.format("delta").load("Files/silver/9_plr_mobilidade_eletrica_cleaned.delta")
    dim_muni = spark.read.format("delta").load("Files/silver/dim_municipality.delta")

In [ ]:
# ============================
    # 2. Normalize & Extract Year
    # ============================
    df_ves = df_ves.withColumn("municipality", upper(trim(col("municipality"))))
    df_ves = df_ves.withColumn("ano", regexp_extract("trimestre", r"(\d{4})", 1))

In [ ]:
df_ves = df_ves.withColumn(
        "date",
        when(col("trimestre").contains("T1"), concat_ws("-", col("ano"), lit("03"), lit("31")))
        .when(col("trimestre").contains("T2"), concat_ws("-", col("ano"), lit("06"), lit("30")))
        .when(col("trimestre").contains("T3"), concat_ws("-", col("ano"), lit("09"), lit("30")))
        .when(col("trimestre").contains("T4"), concat_ws("-", col("ano"), lit("12"), lit("31")))
    )
    df_ves = df_ves.withColumn("date", to_date("date", "yyyy-MM-dd"))
    df_ves = df_ves.withColumn("sk_date", date_format("date", "yyyyMMdd").cast("int"))

In [ ]:
# ============================
    # 3. Normalize PLR and Aggregate
    # ============================
    df_plr = df_plr.withColumn("municipality", upper(trim(col("municipality"))))
    df_plr = df_plr.withColumn("ano", col("ano").cast("string"))

In [ ]:
df_plr_agg = df_plr.groupBy("municipality", "ano") \
        .agg(_sum("pedidos_de_ligacao_a_rede_executados").alias("pedidos_total"))

In [ ]:
# ============================
    # 4. Join VES with PLR
    # ============================
    df_fact = df_ves.join(
        df_plr_agg,
        on=["municipality", "ano"],
        how="left"
    )

In [ ]:
# ============================
    # 5. Join with dim_municipality
    # ============================
    if "sk_municipality" in df_fact.columns:
        df_fact = df_fact.drop("sk_municipality")

In [ ]:
df_fact = df_fact.join(
        dim_muni.select("municipality", "sk_municipality"),
        on="municipality",
        how="left"
    )

In [ ]:
# ============================
    # 6. Select Final Columns
    # ============================
    df_fact_final = df_fact.select(
        "sk_date",
        "sk_municipality",
        "potencia_maxima_admissivel",
        "num_instalacoes",
        "pedidos_total"
    )

In [ ]:
df_fact_final.show(15, truncate=False)

In [ ]:
# ============================
    # 7. Save to Gold Layer and Register as SQL Table
    # ============================

In [ ]:
# ⚠️ Nome canónico para SQL (sem espaços, camel case consistente)
    table_name = "fact_electricmobility"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
# Guardar como Delta (Gold)
    df_fact_final.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .save(path_gold)
    print(f"💾 Saved {table_name} to {path_gold}")

In [ ]:
# Apagar versão anterior da tabela SQL (segurança extra opcional)
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")

In [ ]:
# Registar como SQL Table
    df_fact_final.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable(table_name)
    print(f"✅ SQL Table '{table_name}' registada com sucesso no metastore!")

In [ ]:
{"execution_finish_time":"2025-06-24T20:17:03.6622356Z","execution_start_time":"2025-06-24T20:16:19.7431325Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"36ebc6a3-c330-4cd2-9ea5-23ecd2c6ccab","queued_time":"2025-06-24T20:16:09.3956008Z","session_id":"d80b9515-da52-44ad-b2ce-38e88a008065","session_start_time":"2025-06-24T20:16:09.39655Z","spark_pool":null,"state":"finished","statement_id":3,"statement_ids":[3]}

In [ ]:
+--------+----------------------------------------------------------------+--------------------------+---------------+-------------+
    |sk_date |sk_municipality                                                 |potencia_maxima_admissivel|num_instalacoes|pedidos_total|
    +--------+----------------------------------------------------------------+--------------------------+---------------+-------------+
    |20240630|53c6caee2474904a0b99435a547b095a6b6e1d8965f9308ace7bf60ea1ede6a7|10.35                     |2              |2.0          |
    |20240630|6a16057303e5690a879a6d4e7dee74184379aa5376d4f089a1583285df5087d7|20.7                      |1              |1.0          |
    |20240630|ab6064b5ad9c7d201a6d673c065dbf14f0b3e68a5dc14372b5fb919696046e9e|400.0                     |3              |12.0         |
    |20240630|76517a9e1e8a26395c34dbc7666d1171b14ab32b78bcac8c21eb191757f7859f|20.7                      |4              |1.0          |
    |20240630|3ea0d7cb8c59962370b69b2632ea1f292154407b07cec4c98368410a45beb661|10.35                     |1              |1.0          |
    |20240630|ab6064b5ad9c7d201a6d673c065dbf14f0b3e68a5dc14372b5fb919696046e9e|6.9                       |2              |12.0         |
    |20240630|5921083550017b761ae29ffb4fdf39cdb3da6c19c98c62a3bb31f43d7eafd767|62.62                     |1              |1.0          |
    |20240630|e930b8b50b364f34be9068634cc655b89c3f3bea0272e44892559e326badddff|400.0                     |1              |7.0          |
    |20240630|a7fb70c3249a24ed593850d297538b59b0bcfa5caa21851acb7f910df87fb9fc|200.0                     |1              |NULL         |
    |20240630|a0abb5721879d137f6bc7f13c80866b0ed61e90aef56e39a39ed64a431158ef0|110.0                     |2              |1.0          |
    |20240630|d5f9d748e5f950ffc4a1cd0303b19e1d47e0d3e143067f7a0b1c38f4af9d3f99|13.8                      |1              |3.0          |
    |20240630|01f8f6eb536c2ec47db6bf08999db978076d19ad04d234e7be4886d6fa52531d|10.35                     |5              |NULL         |
    |20240630|92168b946d847a6d96b631ace7968ea84dd70b533e3ec5e6bafe241e8680b7ac|2000.0                    |1              |21.0         |
    |20240630|b14c6347f8bbc7219d389217ee34c9fbb50379fe491382176c6fc0f3f506bbca|6.9                       |1              |4.0          |
    |20240630|89368e7c0071377ace472e98f8f1b717864091a7c2c9aaaadad6c77296cd63dc|140.0                     |2              |19.0         |
    +--------+----------------------------------------------------------------+--------------------------+---------------+-------------+
    only showing top 15 rows

In [ ]:
💾 Saved Fact_ElectricMobility to Files/gold/Fact_ElectricMobility.delta
    ✅ SQL Table 'Fact_ElectricMobility' registada com sucesso no metastore!

In [ ]:
fact_serviceorders

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import (
        col, sha2, concat_ws, to_date, sum as _sum
    )

In [ ]:
# Iniciar Spark session
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# =============================
    # 1. Load Cleaned Silver Dataset
    # =============================
    print("📥 Loading Silver cleaned dataset...")
    df_orders = spark.read.format("delta").load("Files/silver/15_ordens_de_servico_cleaned.delta")
    print("✅ Dataset loaded:")
    df_orders.printSchema()

In [ ]:
# =============================
    # 2. Generate sk_date
    # =============================
    print("📅 Generating sk_date from 'data' column...")
    df_orders = df_orders.withColumn("sk_date", sha2(col("data").cast("string"), 256))

In [ ]:
# =============================
    # 3. Join with dim_servicetype
    # =============================
    print("🔗 Joining with dim_servicetype...")
    dim_service = spark.read.table("dim_servicetype")

In [ ]:
df_orders = df_orders.join(
        dim_service,
        df_orders["tipo_de_servico"] == dim_service["service_type"],
        how="left"
    ).drop("service_type")

In [ ]:
# =============================
    # 4. Clean and Cast
    # =============================
    print("🧼 Filtering non-null keys and casting column...")
    df_orders = df_orders.filter(
        col("sk_date").isNotNull() &
        col("sk_municipality").isNotNull() &
        col("sk_service_type").isNotNull()
    )

In [ ]:
df_orders = df_orders.withColumn("service_orders", col("ordens_servico_realizadas").cast("int"))

In [ ]:
# =============================
    # 5. Aggregate
    # =============================
    print("📊 Aggregating by sk_date, sk_municipality, sk_service_type...")
    fact_df = df_orders.groupBy(
        "sk_date", "sk_municipality", "sk_service_type"
    ).agg(
        _sum("service_orders").alias("service_orders")
    )

In [ ]:
# =============================
    # 6. Preview
    # =============================
    print("📋 Preview of fact_serviceorders:")
    fact_df.printSchema()
    fact_df.show(10, truncate=False)

In [ ]:
# =============================
    # 7. Save and Register as SQL Table
    # =============================
    table_name = "fact_serviceorders"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
print(f"💾 Saving to Gold: {path_gold}")
    fact_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(path_gold)

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    fact_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name)
    print(f"✅ SQL Table '{table_name}' registered and ready for Power BI.")

In [ ]:
{"execution_finish_time":"2025-06-24T20:33:23.9452703Z","execution_start_time":"2025-06-24T20:33:09.7380561Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"e18e0de6-2b9b-4a45-8190-4d779610a53b","queued_time":"2025-06-24T20:33:09.7368166Z","session_id":"d80b9515-da52-44ad-b2ce-38e88a008065","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":11,"statement_ids":[11]}

In [ ]:
📥 Loading Silver cleaned dataset...
    ✅ Dataset loaded:
    root
     |-- ano: string (nullable = true)
     |-- mes: string (nullable = true)
     |-- data: string (nullable = true)
     |-- distrito: string (nullable = true)
     |-- concelho: string (nullable = true)
     |-- freguesia: string (nullable = true)
     |-- tipo_de_servico: string (nullable = true)
     |-- ordens_servico_realizadas: string (nullable = true)
     |-- coddistrito: string (nullable = true)
     |-- coddistritoconcelho: string (nullable = true)
     |-- coddistritoconcelhofreguesia: string (nullable = true)
     |-- district: string (nullable = true)
     |-- municipality: string (nullable = true)
     |-- parish: string (nullable = true)
     |-- sk_district: string (nullable = true)
     |-- sk_municipality: string (nullable = true)
     |-- sk_parish: string (nullable = true)

In [ ]:
📅 Generating sk_date from 'data' column...
    🔗 Joining with Dim_ServiceType...
    🧼 Filtering non-null keys and casting column...
    📊 Aggregating by sk_date, sk_municipality, sk_service_type...
    📋 Preview of Fact_ServiceOrders:
    root
     |-- sk_date: string (nullable = true)
     |-- sk_municipality: string (nullable = true)
     |-- sk_service_type: string (nullable = true)
     |-- service_orders: long (nullable = true)

In [ ]:
+----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+--------------+
    |sk_date                                                         |sk_municipality                                                 |sk_service_type                                                 |service_orders|
    +----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+--------------+
    |6cd8ddf7f7a4fc382af87d16f49e88bf58064ea01591ff6a92938a4e95ccf402|2b2d28eb3a01571f11049359e9fd47db8adb5ef6884c4dbed4cf5b8abce60019|94e04b4fc48c1b318c8b370e1d593ba3bc1aebfae7cdc1bf0e0724dd6a0b1e0d|17            |
    |bef719459537b739566cb2fe1f0f24582c87e21d88e765ca3c50c1291358e83c|16e003285eccfe5f9ee211c782d705bad241f04726cf6c03511dc337d3dbaafb|d6af5440a41f096431c0bd98734177aa19d8caa2c312302359ff2b0577cd8833|38            |
    |049cebe321997964fecde9133bfd81abeb4dd400666d74c9d00c646d3d80d99c|4f15f22f7f1817e152a723dbd734589656ab000098a9a14e8c37a5677a09fc33|b5b2e21052959a7facd75e38a8ef0b007af04d19b43ebed464a7790004ef8a92|192           |
    |cb4aa8fc5c0c032127baadde1a3200d56b3f9218072e7692abc35fb987e4113e|8d24b48f7084913dd59bc8d6001b71f3c194122cf0dbab5ae46e7e28e1bf787a|816fd6b3076b7ce629cfcad5c1ab9a9100fd6b5def8134c333bbc66e6324efab|127           |
    |bddf8a632f93747c50673f53cc770ff2b18e3729e5af3182a1aa1b5cb17fa033|c2f60e07f6b618749f21a498803929582f2aa416685c1257641a572142fa65a5|94e04b4fc48c1b318c8b370e1d593ba3bc1aebfae7cdc1bf0e0724dd6a0b1e0d|384           |
    |97ad1ea819fcf2b8876537bd8d56e98183629212cc595846fe2fb581e1e94282|32d3cce444a89db60920aa196e93215f613116d8d0a82f2ed0304c93a4c44150|b5b2e21052959a7facd75e38a8ef0b007af04d19b43ebed464a7790004ef8a92|10            |
    |55f2d845ac24cbff201ee7f4e3ca5b67e36ed179a833c2b19b1df35242d7cb7f|2cb7224de4a596c18698e77d9610087ef55e30150733274c36a6604f8ca78989|b5b2e21052959a7facd75e38a8ef0b007af04d19b43ebed464a7790004ef8a92|126           |
    |76d94d353572d4f2ec20eeb9a871590f4450e7594ba5e456414dcdfdb8a3a2fe|eb7cee3260c054e88dfffbd05d02aa3fadb0e75474488a57627f49c30ca87cd9|94e04b4fc48c1b318c8b370e1d593ba3bc1aebfae7cdc1bf0e0724dd6a0b1e0d|18            |
    |bddf8a632f93747c50673f53cc770ff2b18e3729e5af3182a1aa1b5cb17fa033|2c6a017fa6912d22c7edbbc2c74423d5d471dc7b6fce9ab45543f4dda713e6df|0c283ec723be22c4e1375efe100a8f3ef9a9c47b64f79dc6a19e9dde91eed560|64            |
    |4e3d7e6e2a00e6b03010854139265f7143cab652ab02c6bb2fe2319f3ee359f4|a0abb5721879d137f6bc7f13c80866b0ed61e90aef56e39a39ed64a431158ef0|612e12fbebae460b3c5ef70845bcbe74c6eadf3e1d4a701649de7241f28f57f3|221           |
    +----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+--------------+
    only showing top 10 rows

In [ ]:
💾 Saving to Gold: Files/gold/Fact_ServiceOrders.delta
    ✅ SQL Table 'Fact_ServiceOrders' registered and ready for Power BI.

In [ ]:
fact_energyproduction

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, sum as _sum

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Cleaned Silver Table
    # ============================
    df_silver = spark.read.format("delta").load("Files/silver/energia_produzida_total_nacional_cleaned.delta")
    print("✅ Loaded cleaned Silver table: energia_produzida_total_nacional_cleaned")

In [ ]:
# ============================
    # 2. Aggregate by sk_date
    # ============================
    df_gold = df_silver.groupBy("sk_date").agg(
        _sum("energy_production").alias("total_energy_production")
    )
    print("🔢 Aggregated total energy production per day")

In [ ]:
# ============================
    # 3. Preview Result
    # ============================
    df_gold.show(10, truncate=False)

In [ ]:
# ============================
    # 4. Save to Gold and Register SQL Table
    # ============================
    table_name = "fact_energyproduction"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
# Guardar como Delta na Gold
    df_gold.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .save(path_gold)
    print(f"💾 Saved {table_name} to Gold path: {path_gold}")

In [ ]:
# Registar como SQL Table visível
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    df_gold.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable(table_name)
    print(f"✅ SQL Table '{table_name}' registada com sucesso!")

In [ ]:
{"execution_finish_time":"2025-06-24T20:20:21.8696445Z","execution_start_time":"2025-06-24T20:19:41.8614995Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"fb28a030-bb23-4b9b-80d2-e09876027325","queued_time":"2025-06-24T20:19:41.8602862Z","session_id":"d80b9515-da52-44ad-b2ce-38e88a008065","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":4,"statement_ids":[4]}

In [ ]:
✅ Loaded cleaned Silver table: energia_produzida_total_nacional_cleaned
    🔢 Aggregated total energy production per day
    +--------+-----------------------+
    |sk_date |total_energy_production|
    +--------+-----------------------+
    |20240118|1.647197995E8          |
    |20240212|1.4300477801000002E8   |
    |20240908|1.15137369456E8        |
    |20250528|1.4107877587699997E8   |
    |20230530|1.3733759175E8         |
    |20240710|1.4506698552599996E8   |
    |20240406|1.2918431175E8         |
    |20240904|1.40679526868E8        |
    |20241025|1.4176189584099996E8   |
    |20250201|1.5216408574699998E8   |
    +--------+-----------------------+
    only showing top 10 rows

In [ ]:
💾 Saved Fact_EnergyProduction to Gold path: Files/gold/Fact_EnergyProduction.delta
    ✅ SQL Table 'Fact_EnergyProduction' registada com sucesso!

In [ ]:
dim_zipcode

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, sha2, trim

In [ ]:
# ============================
    # 0. Start Spark session
    # ============================
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Read Raw CTT Dataset
    # ============================
    print("📥 Reading CTT base file...")
    ctt_path = "Files/bronze/CTT/todos_cp.txt"
    df_ctt = spark.read.option("delimiter", ";").csv(ctt_path)

In [ ]:
print("✅ Raw dataset loaded:")
    df_ctt.printSchema()

In [ ]:
# ============================
    # 2. Select and Rename Columns
    # ============================
    print("🧾 Selecting and renaming columns...")
    zipcodes_df = (
        df_ctt.select(
            col("_c14").alias("zipcode"),
            col("_c16").alias("postal_label")
        )
        .filter(col("zipcode").isNotNull())
        .withColumn("zipcode", trim(col("zipcode")))
    )

In [ ]:
# ============================
    # 3. Generate Surrogate Key
    # ============================
    print("🔐 Generating surrogate key 'sk_zipcode'...")
    zipcodes_df = zipcodes_df.withColumn("sk_zipcode", sha2(col("zipcode"), 256))

In [ ]:
# ============================
    # 4. Drop Duplicates
    # ============================
    print("🧼 Removing duplicate zipcodes...")
    zipcodes_df = zipcodes_df.dropDuplicates(["zipcode"])

In [ ]:
# ============================
    # 5. Preview Result
    # ============================
    print("✅ Preview of dim_zipcode:")
    zipcodes_df.show(10, truncate=False)

In [ ]:
# ============================
    # 6. Save as Delta and SQL Table
    # ============================
    print("💾 Saving dim_zipcode as Delta file and SQL table...")
    zipcodes_df.write.mode("overwrite").format("delta").save("Files/silver/dim_zipcode.delta")

In [ ]:
spark.sql("DROP TABLE IF EXISTS dim_zipcode")
    zipcodes_df.write.mode("overwrite").format("delta").saveAsTable("dim_zipcode")

In [ ]:
print("✅ dim_zipcode created and saved successfully.")

In [ ]:
{"execution_finish_time":"2025-06-25T10:53:04.7276204Z","execution_start_time":"2025-06-25T10:52:11.236289Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"ae501429-d430-4489-9956-c90892914264","queued_time":"2025-06-25T10:51:58.0888143Z","session_id":"ba50ac19-23fd-4b0a-8146-282a48c20351","session_start_time":"2025-06-25T10:51:58.0898915Z","spark_pool":null,"state":"finished","statement_id":3,"statement_ids":[3]}

In [ ]:
📥 Reading CTT base file...
    ✅ Raw dataset loaded:
    root
     |-- _c0: string (nullable = true)
     |-- _c1: string (nullable = true)
     |-- _c2: string (nullable = true)
     |-- _c3: string (nullable = true)
     |-- _c4: string (nullable = true)
     |-- _c5: string (nullable = true)
     |-- _c6: string (nullable = true)
     |-- _c7: string (nullable = true)
     |-- _c8: string (nullable = true)
     |-- _c9: string (nullable = true)
     |-- _c10: string (nullable = true)
     |-- _c11: string (nullable = true)
     |-- _c12: string (nullable = true)
     |-- _c13: string (nullable = true)
     |-- _c14: string (nullable = true)
     |-- _c15: string (nullable = true)
     |-- _c16: string (nullable = true)

In [ ]:
🧾 Selecting and renaming columns...
    🔐 Generating surrogate key 'sk_zipcode'...
    🧼 Removing duplicate zipcodes...
    ✅ Preview of DimZipCode:
    +-------+------------+----------------------------------------------------------------+
    |zipcode|postal_label|sk_zipcode                                                      |
    +-------+------------+----------------------------------------------------------------+
    |1000   |LISBOA      |40510175845988f13f6162ed8526f0b09f73384467fa855e1e79b44a56562a58|
    |1003   |LISBOA      |8c9a013ab70c0434313e3e881c310b9ff24aff1075255ceede3f2c239c231623|
    |1049   |LISBOA      |0c62cc42d6479a691f03083654ab6a7a84229ab156c948ba8d3b6c79ddd95536|
    |1050   |LISBOA      |ffa6059b954a4602a9fa1518d10ca6163bce3f9d4bd3ee51c860eb6c2da16675|
    |1053   |LISBOA      |285b71922aaa01d870483d3b4f59e4a61c2057e1476854a810f387a1d2317806|
    |1067   |LISBOA      |f48280c0107726b99afcbd3cd0a46a2bac5f1ddb44e8768f232ac4da562eb166|
    |1068   |LISBOA      |0f0b82fae280ae9fec1905f029b6ee9a9c85bb6cc5151da6dafe38a7902a4a53|
    |1069   |LISBOA      |2ff2a721fab172f1ab7e77774978c698899249076747675c872eaa9b2f6780b8|
    |1070   |LISBOA      |d0f451801443fa1478e1cf080e8d63dead650c9712b2631edf7e1658a98967d9|
    |1073   |LISBOA      |be1722b1d104ffb2753260682954754d9757b986013823c5df20f28e0f74e655|
    +-------+------------+----------------------------------------------------------------+
    only showing top 10 rows

In [ ]:
💾 Saving DimZipCode as Delta file and SQL table...
    ✅ DimZipCode created and saved successfully.

In [ ]:
dim_district, dim_municipality and dim_parish

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, sha2, concat_ws, upper, trim

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 🔧 Função auxiliar
    # ============================
    def normalize_str(c):
        return upper(trim(col(c)))

In [ ]:
# ============================
    # 📍 dim_district
    # ============================
    print("\n📥 Carregando dados para dim_district...")

In [ ]:
df_dist = (
        spark.read.option("header", True).csv("Files/bronze/CAOP/Areas_Distritos_CAOP2024.csv")
             .withColumn("district", normalize_str("distrito_ilha_dsg"))
             .withColumn("sk_district", sha2(col("district"), 256))
             .select("sk_district", "district", "area_ha", "area_km2")
    )

In [ ]:
print("📋 Preview dim_district:")
    df_dist.show(10, truncate=False)

In [ ]:
# Guardar e registar
    gold_path_dist = "Files/gold/dim_district.delta"
    df_dist.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(gold_path_dist)

In [ ]:
spark.sql("DROP TABLE IF EXISTS dim_district")
    df_dist.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("dim_district")
    print("✅ dim_district registada e visível no Power BI.\n")

In [ ]:
# ============================
    # 🏙️ dim_municipality
    # ============================
    print("📥 Carregando dados para dim_municipality...")

In [ ]:
df_muni_base = (
        spark.read.option("header", True)
                  .csv("Files/bronze/CAOP/Areas_Municipios_CAOP2024.csv")
                  .withColumn("district", normalize_str("distrito_ilha_dsg"))
                  .withColumn("municipality", normalize_str("município_dsg"))
                  .withColumn("sk_municipality", sha2(concat_ws("-", "district", "municipality"), 256))
    )

In [ ]:
df_muni = df_muni_base.select("sk_municipality", "district", "municipality", "area_ha", "area_km2")

In [ ]:
df_codconcelho = spark.read.format("delta").load("Files/silver/dim_codconcelho.delta")

In [ ]:
df_muni_joined = (
        df_muni.alias("m")
        .join(df_codconcelho.select("sk_municipality", "cod_concelho").alias("c"), on="sk_municipality", how="left")
    )

In [ ]:
print("📋 Preview dim_municipality:")
    df_muni_joined.show(10, truncate=False)

In [ ]:
# Guardar e registar
    gold_path_muni = "Files/gold/dim_municipality.delta"
    df_muni_joined.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(gold_path_muni)

In [ ]:
spark.sql("DROP TABLE IF EXISTS dim_municipality")
    df_muni_joined.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("dim_municipality")
    print("✅ dim_municipality registada com cod_concelho e visível no Power BI.\n")

In [ ]:
# ============================
    # 🏡 dim_parish (com sk_municipality)
    # ============================
    print("📥 Carregando dados para dim_parish (com sk_municipality)...")

In [ ]:
df_freg = (
        spark.read.option("header", True).csv("Files/bronze/CAOP/Areas_Freguesias_CAOP2024.csv")
             .withColumn("district", normalize_str("distrito_ilha_dsg"))
             .withColumn("municipality", normalize_str("município_dsg"))
             .withColumn("parish", normalize_str("freguesia_dsg"))
             .withColumn("sk_parish", sha2(concat_ws("-", "district", "municipality", "parish"), 256))
             .withColumn("sk_municipality", sha2(concat_ws("-", "district", "municipality"), 256))
             .select("sk_parish", "sk_municipality", "district", "municipality", "parish", "area_ha", "area_km2")
    )

In [ ]:
print("📋 Preview dim_parish:")
    df_freg.show(10, truncate=False)

In [ ]:
# Guardar e registar
    gold_path_parish = "Files/gold/dim_parish.delta"
    df_freg.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(gold_path_parish)

In [ ]:
spark.sql("DROP TABLE IF EXISTS dim_parish")
    df_freg.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("dim_parish")
    print("✅ dim_parish registada com sucesso e visível no Power BI.\n")

In [ ]:
{"execution_finish_time":"2025-06-28T14:04:22.6237978Z","execution_start_time":"2025-06-28T14:04:01.1429421Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"748c0154-5472-4ad8-8915-6274f4920861","queued_time":"2025-06-28T14:04:01.1416238Z","session_id":"ccfaf4ac-d020-4f44-a99a-588f8e59ebfd","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":16,"statement_ids":[16]}

In [ ]:
📥 Carregando dados para DimDistrict...
    📋 Preview DimDistrict:
    +----------------------------------------------------------------+--------------+----------+--------+
    |sk_district                                                     |district      |area_ha   |area_km2|
    +----------------------------------------------------------------+--------------+----------+--------+
    |a68e0baf3c58a0820839e02c76c0053269760def8fd8877f80a70aa50e04bdde|AVEIRO        |280094.16 |2800.94 |
    |bbe2c7e9b65b8a141abd7922b9ca4bb7f1686bb434f84d752b87ec39b438b74d|BEJA          |1026332.49|10263.32|
    |f90f638e94c8765dd28e203431c619e4c44d9a0b2d46deeb8b30ab72a66fb7de|BRAGA         |270615.05 |2706.15 |
    |d117c8c3e1cf3e067a547c6ec93d367785d5902d7e28701a2f237fdaa48930fd|BRAGANÇA      |659853.71 |6598.54 |
    |c622b2c708ecbf758535fcce1d476cfa38da23e6467db558ac4268b4474793b5|CASTELO BRANCO|662747.31 |6627.47 |
    |12922d39abe57f8d849198e4c1dde253dee2efca3750e8a4b5235135b76b16c2|COIMBRA       |397373.28 |3973.73 |
    |5f94947f0692a8fd51d364b3f306a0ec5301c5938a7065f5de9b35403373e9d0|ÉVORA         |739346.41 |7393.46 |
    |b2013a67571ea967907a2dda89fc05dd540ec1c6a9a964b1c64ab3128ad6fd12|FARO          |499679.49 |4996.79 |
    |27bc243bbf1a34e65201a4c56ff081936979bbcffaafec9f48bc3afb98cf367f|GUARDA        |553530.81 |5535.31 |
    |5f480a0ac8af6bdb3830d6231ab99bedb0dbec8048f8b48088af5b37f1f78f14|LEIRIA        |350578.68 |3505.79 |
    +----------------------------------------------------------------+--------------+----------+--------+
    only showing top 10 rows

In [ ]:
✅ DimDistrict registada e visível no Power BI.

In [ ]:
📥 Carregando dados para DimMunicipality...
    📋 Preview DimMunicipality:
    +----------------------------------------------------------------+--------+--------------------+--------+--------+------------+
    |sk_municipality                                                 |district|municipality        |area_ha |area_km2|cod_concelho|
    +----------------------------------------------------------------+--------+--------------------+--------+--------+------------+
    |3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA              |33527.44|335.27  |0101        |
    |4664effdfd10c006c69d0d80d7e9218e241cef39fdc2a66e561f34908290c411|AVEIRO  |ALBERGARIA-A-VELHA  |15882.50|158.83  |0102        |
    |f60534b5361a3a29fc1b35985d4dba70acb42e4b65f7315c82aacc2e79b59792|AVEIRO  |ANADIA              |21663.48|216.63  |0103        |
    |bc4ac87787cff7c77c67b55a4f041a511d82bcd8093306ff54432e7d904a88e6|AVEIRO  |AROUCA              |32910.52|329.11  |0104        |
    |771924b54dc8ac992fb02ac943590380e9a44aa62e6612bc27143db13de44667|AVEIRO  |AVEIRO              |19757.57|197.58  |0105        |
    |d6b3689721792c1365db9604e7ab5f2b6caef54fb6bb940da8c7fc98f4dbe11b|AVEIRO  |CASTELO DE PAIVA    |11500.53|115.01  |0106        |
    |892ee65acf52aa7de0ad5699e05575ca01bf0add6c7cb74695ac4c4f70346d49|AVEIRO  |ESPINHO             |2105.68 |21.06   |0107        |
    |a3a217cf4b1ae10a1e9033bd9e6064bee158dca3163fb3789601cd11bb073eb1|AVEIRO  |ESTARREJA           |10817.33|108.17  |0108        |
    |9e5efd0dd2f50c33247e2d56ff8f3b2025cdb275201aeae7d36a5e4ca77af074|AVEIRO  |SANTA MARIA DA FEIRA|21587.65|215.88  |0109        |
    |484bfffc86381b0ba75a7e9335f316ff2abeca62605979e34fb9c76b296f5f2e|AVEIRO  |ÍLHAVO              |7347.55 |73.48   |0110        |
    +----------------------------------------------------------------+--------+--------------------+--------+--------+------------+
    only showing top 10 rows

In [ ]:
✅ DimMunicipality registada com cod_concelho e visível no Power BI.

In [ ]:
📥 Carregando dados para DimParish (com sk_municipality)...
    📋 Preview DimParish:
    +----------------------------------------------------------------+----------------------------------------------------------------+--------+------------+------------------------------------------------------------------------+-------+--------+
    |sk_parish                                                       |sk_municipality                                                 |district|municipality|parish                                                                  |area_ha|area_km2|
    +----------------------------------------------------------------+----------------------------------------------------------------+--------+------------+------------------------------------------------------------------------+-------+--------+
    |b17a6d4d8ffa115b8eb27d40d3c066e8dd3b81bc5d27ccc1f8ef576437ba868e|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |AGUADA DE CIMA                                                          |2839.31|28.39   |
    |1962f0904f21f8d7fedee020a8b8cc292ec1ccc2d724c665b3bbcb0275bbb359|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |FERMENTELOS                                                             |858.20 |8.58    |
    |0ccb08873e78309e306f745ee37b6f592478f3317fdb242b69881f7ec6485b29|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |MACINHATA DO VOUGA                                                      |3195.44|31.95   |
    |706112d2497e98a124f0581edfe4175b30390392a9a4371e07ac1fe3914b640a|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |VALONGO DO VOUGA                                                        |4340.48|43.40   |
    |b0c603c37d447c952b341520db07b410e26b69d1671603c80641c10c1bea6954|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |UNIÃO DAS FREGUESIAS DE ÁGUEDA E BORRALHA                               |3602.93|36.03   |
    |abb4d9885fc4ccc423ff4e8a572a9678c6e710ca451ecf885c3adf96d724c347|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |UNIÃO DAS FREGUESIAS DE BARRÔ E AGUADA DE BAIXO                         |1019.01|10.19   |
    |27047c9eeaeb54353fb7900fb192868e9b9d372dcd2404d031492b5b0f4d8677|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |UNIÃO DAS FREGUESIAS DE BELAZAIMA DO CHÃO, CASTANHEIRA DO VOUGA E AGADÃO|8809.03|88.09   |
    |60a0be2f5841bc1723ff9bb4a431a0acd9293352d277ba77a1ebe27dfc063c09|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |UNIÃO DAS FREGUESIAS DE RECARDÃES E ESPINHEL                            |1991.78|19.92   |
    |53d6eb86d61c1da9d1feee46d992f2ff6d22584ce79fb98840a665c2bf55ccab|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |UNIÃO DAS FREGUESIAS DE TRAVASSÔ E ÓIS DA RIBEIRA                       |1112.19|11.12   |
    |689e230e020c86dc1f4729957fdba6d4c9ebcd190577bf998a9e8848f0d31b30|3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|AVEIRO  |ÁGUEDA      |UNIÃO DAS FREGUESIAS DE TROFA, SEGADÃES E LAMAS DO VOUGA                |1586.43|15.86   |
    +----------------------------------------------------------------+----------------------------------------------------------------+--------+------------+------------------------------------------------------------------------+-------+--------+
    only showing top 10 rows

In [ ]:
✅ DimParish registada com sucesso e visível no Power BI.

In [ ]:
dim_servicetype

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import sha2, upper, trim, col

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
print("📥 Loading and preparing service types from Silver cleaned dataset...")

In [ ]:
# Lê o dataset Silver limpo
    df_cleaned = spark.read.table("15_ordens_de_servico_cleaned")

In [ ]:
# Prepara a dimensão
    dim_service_type = (
        df_cleaned
        .select("tipo_de_servico")
        .filter(col("tipo_de_servico").isNotNull() & (col("tipo_de_servico") != ""))
        .withColumn("service_type", upper(trim(col("tipo_de_servico"))))
        .select("service_type")
        .distinct()
        .withColumn("sk_service_type", sha2(col("service_type"), 256))
        .select("sk_service_type", "service_type")
    )

In [ ]:
# ============================
    # Guardar em Delta e Registar como Tabela SQL
    # ============================
    table_name = "dim_servicetype"
    gold_path = f"Files/gold/{table_name}.delta"

In [ ]:
# Guardar camada Gold
    print("💾 Saving to Gold layer...")
    dim_service_type.write \
        .mode("overwrite") \
        .format("delta") \
        .option("overwriteSchema", "true") \
        .save(gold_path)

In [ ]:
# Registar como tabela SQL (usando saveAsTable)
    print("🗃️ Registering as SQL table...")
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    dim_service_type.write \
        .mode("overwrite") \
        .format("delta") \
        .option("overwriteSchema", "true") \
        .saveAsTable(table_name)

In [ ]:
print(f"✅ SQL Table '{table_name}' created and registered successfully.")

In [ ]:
# ============================
    # Visualizar resultado
    # ============================
    print("📑 Schema da dim_servicetype:")
    dim_service_type.printSchema()

In [ ]:
print("🔍 Preview da Dimensão:")
    dim_service_type.show(truncate=False)

In [ ]:
{"execution_finish_time":"2025-06-24T21:24:21.1491132Z","execution_start_time":"2025-06-24T21:24:11.4444518Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"7a67c614-ae90-468b-9344-23988fd925a9","queued_time":"2025-06-24T21:24:11.4431872Z","session_id":"ca3fa664-6974-488f-9f07-58f4d85e0164","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":7,"statement_ids":[7]}

In [ ]:
📥 Loading and preparing service types from Silver cleaned dataset...
    💾 Saving to Gold layer...
    🗃️ Registering as SQL table...
    ✅ SQL Table 'Dim_ServiceType' created and registered successfully.
    📑 Schema da Dim_ServiceType:
    root
     |-- sk_service_type: string (nullable = true)
     |-- service_type: string (nullable = true)

In [ ]:
🔍 Preview da Dimensão:
    +----------------------------------------------------------------+----------------------------------------+
    |sk_service_type                                                 |service_type                            |
    +----------------------------------------------------------------+----------------------------------------+
    |612e12fbebae460b3c5ef70845bcbe74c6eadf3e1d4a701649de7241f28f57f3|ALTERAÇÃO CONTRATUAL                    |
    |2e54232b6f68fc950759a11226d25ba051691934c07c4a09d8ba82b68391a9d9|ASSISTÊNCIA A CLIENTES                  |
    |f4cb9bfa9d112858221c5bebd458316732ca5464c13c0732df6a85fa2cb6e886|RESTABELECIMENTOS                       |
    |94e04b4fc48c1b318c8b370e1d593ba3bc1aebfae7cdc1bf0e0724dd6a0b1e0d|REDUÇÕES TEMPORÁRIAS POTÊNCIA CONTRATADA|
    |b5b2e21052959a7facd75e38a8ef0b007af04d19b43ebed464a7790004ef8a92|REPOSIÇÕES POTÊNCIA CONTRATADA          |
    |0c283ec723be22c4e1375efe100a8f3ef9a9c47b64f79dc6a19e9dde91eed560|ATIVAÇÕES                               |
    |ce6cd494327fdb42d173a9cf312294b6415501484b214f345834e824c6e91a17|SERVIÇOS DIVERSOS                       |
    |23812a933d8075c9164e0626adaca493ab14056323e81f304506aaad4bad571b|DESATIVAÇÕES                            |
    |816fd6b3076b7ce629cfcad5c1ab9a9100fd6b5def8134c333bbc66e6324efab|MANUTENÇÃO                              |
    |d6af5440a41f096431c0bd98734177aa19d8caa2c312302359ff2b0577cd8833|INTERRUPÇÕES                            |
    +----------------------------------------------------------------+----------------------------------------+

In [ ]:
dim_servicetype

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import sha2, upper, trim, col

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Silver Cleaned Dataset
    # ============================
    df_cleaned = spark.read.table("15_ordens_de_servico_cleaned")
    print("📥 Loaded Silver cleaned dataset: 15_ordens_de_servico_cleaned")

In [ ]:
# ============================
    # 2. Gerar valores únicos e normalizados
    # ============================
    dim_service_type = (
        df_cleaned
        .select("tipo_de_servico")
        .filter(col("tipo_de_servico").isNotNull() & (col("tipo_de_servico") != ""))
        .withColumn("service_type", upper(trim(col("tipo_de_servico"))))
        .select("service_type")
        .distinct()
        .withColumn("sk_service_type", sha2(col("service_type"), 256))
        .select("sk_service_type", "service_type")
    )

In [ ]:
# ============================
    # 3. Preview da Tabela
    # ============================
    print("🔍 Preview da dim_servicetype:")
    dim_service_type.show(10, truncate=False)

In [ ]:
# ============================
    # 4. Guardar e Registar SQL Table
    # ============================
    table_name = "dim_servicetype"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
# Guardar como Delta (Gold Layer)
    dim_service_type.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .save(path_gold)

In [ ]:
print(f"💾 Saved {table_name} to Gold at {path_gold}")

In [ ]:
# Registar como SQL Table no Fabric
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")

In [ ]:
dim_service_type.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable(table_name)

In [ ]:
print(f"✅ SQL Table '{table_name}' registada com sucesso no Fabric")

In [ ]:
{"execution_finish_time":"2025-06-24T21:17:16.6513271Z","execution_start_time":"2025-06-24T21:17:04.1301502Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"6c36d03b-30bb-4182-8319-f3abe7571cba","queued_time":"2025-06-24T21:17:04.1289447Z","session_id":"ca3fa664-6974-488f-9f07-58f4d85e0164","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":4,"statement_ids":[4]}

In [ ]:
📥 Loaded Silver cleaned dataset: 15_ordens_de_servico_cleaned
    🔍 Preview da Dim_ServiceType:
    +----------------------------------------------------------------+----------------------------------------+
    |sk_service_type                                                 |service_type                            |
    +----------------------------------------------------------------+----------------------------------------+
    |612e12fbebae460b3c5ef70845bcbe74c6eadf3e1d4a701649de7241f28f57f3|ALTERAÇÃO CONTRATUAL                    |
    |2e54232b6f68fc950759a11226d25ba051691934c07c4a09d8ba82b68391a9d9|ASSISTÊNCIA A CLIENTES                  |
    |f4cb9bfa9d112858221c5bebd458316732ca5464c13c0732df6a85fa2cb6e886|RESTABELECIMENTOS                       |
    |94e04b4fc48c1b318c8b370e1d593ba3bc1aebfae7cdc1bf0e0724dd6a0b1e0d|REDUÇÕES TEMPORÁRIAS POTÊNCIA CONTRATADA|
    |b5b2e21052959a7facd75e38a8ef0b007af04d19b43ebed464a7790004ef8a92|REPOSIÇÕES POTÊNCIA CONTRATADA          |
    |0c283ec723be22c4e1375efe100a8f3ef9a9c47b64f79dc6a19e9dde91eed560|ATIVAÇÕES                               |
    |ce6cd494327fdb42d173a9cf312294b6415501484b214f345834e824c6e91a17|SERVIÇOS DIVERSOS                       |
    |23812a933d8075c9164e0626adaca493ab14056323e81f304506aaad4bad571b|DESATIVAÇÕES                            |
    |816fd6b3076b7ce629cfcad5c1ab9a9100fd6b5def8134c333bbc66e6324efab|MANUTENÇÃO                              |
    |d6af5440a41f096431c0bd98734177aa19d8caa2c312302359ff2b0577cd8833|INTERRUPÇÕES                            |
    +----------------------------------------------------------------+----------------------------------------+

In [ ]:
💾 Saved Dim_ServiceType to Gold at Files/gold/Dim_ServiceType.delta
    ✅ SQL Table 'Dim_ServiceType' registada com sucesso no Fabric

In [ ]:
dim_policy

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import sha2, col, to_date
    from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # STEP 1: Define static policies (inclui nova política)
    # ============================
    data = [
        {
            "policy_name": "Plano Nacional de Mobilidade Elétrica 2023",
            "policy_type": "Mobilidade Elétrica",
            "description": "Plano para acelerar a instalação de postos de carregamento EV.",
            "scope": "Nacional",
            "start_date": "2023-01-01",
            "end_date": "2025-12-31",
            "source_url": "https://www.fundoambiental.pt"
        },
        {
            "policy_name": "Incentivo ao Autoconsumo 2022",
            "policy_type": "Energia Renovável",
            "description": "Incentivo para instalações de produção fotovoltaica doméstica.",
            "scope": "Nacional",
            "start_date": "2022-04-01",
            "end_date": "2024-12-31",
            "source_url": "https://www.dgeg.gov.pt"
        },
        {
            "policy_name": "Plano E-REDES Smart Metering",
            "policy_type": "Digitalização",
            "description": "Plano para massificação de contadores inteligentes na rede.",
            "scope": "E-REDES",
            "start_date": "2021-01-01",
            "end_date": "2025-12-31",
            "source_url": "https://www.e-redes.pt"
        },
        {
            "policy_name": "Injeção Nacional Renovável",
            "policy_type": "Energia Renovável",
            "description": "Medida transversal de produção renovável injetada na rede nacional.",
            "scope": "Nacional",
            "start_date": "2020-01-01",
            "end_date": "2025-12-31",
            "source_url": "https://www.erse.pt"
        }
    ]

In [ ]:
schema = StructType([
        StructField("policy_name", StringType(), False),
        StructField("policy_type", StringType(), False),
        StructField("description", StringType(), True),
        StructField("scope", StringType(), True),
        StructField("start_date", StringType(), True),
        StructField("end_date", StringType(), True),
        StructField("source_url", StringType(), True),
    ])

In [ ]:
# ============================
    # STEP 2: Create DataFrame & Generate SK
    # ============================
    df_policy = spark.createDataFrame(data, schema) \
        .withColumn("start_date", to_date(col("start_date"), "yyyy-MM-dd")) \
        .withColumn("end_date", to_date(col("end_date"), "yyyy-MM-dd")) \
        .withColumn("sk_policy", sha2(col("policy_name"), 256)) \
        .select("sk_policy", "policy_name", "policy_type", "description", "scope", "start_date", "end_date", "source_url")

In [ ]:
# ============================
    # STEP 3: Preview
    # ============================
    print("📑 Schema da dim_policy:")
    df_policy.printSchema()

In [ ]:
print("🔍 Preview da dimensão:")
    df_policy.show(truncate=False)

In [ ]:
# ============================
    # STEP 4: Save as Delta (Gold)
    # ============================
    path_gold = "Files/gold/dim_policy.delta"
    print("💾 Saving as Delta in Gold layer...")
    df_policy.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(path_gold)

In [ ]:
# ============================
    # STEP 5: Register SQL Table
    # ============================
    print("🗃️ Registering as SQL Table...")
    spark.sql("DROP TABLE IF EXISTS dim_policy")
    df_policy.write.mode("overwrite").format("delta").saveAsTable("dim_policy")

In [ ]:
print("✅ SQL Table 'dim_policy' created and registered with success.")

In [ ]:
{"execution_finish_time":"2025-06-27T23:11:26.2876313Z","execution_start_time":"2025-06-27T23:10:53.3670272Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"a7b33f76-50e6-4ebc-8f46-e5b3168084f5","queued_time":"2025-06-27T23:10:43.3235589Z","session_id":"62f3a26e-05ae-4dde-b7d2-bf31a7a8f0f0","session_start_time":"2025-06-27T23:10:43.3247944Z","spark_pool":null,"state":"finished","statement_id":3,"statement_ids":[3]}

In [ ]:
📑 Schema da Dim_Policy:
    root
     |-- sk_policy: string (nullable = true)
     |-- policy_name: string (nullable = false)
     |-- policy_type: string (nullable = false)
     |-- description: string (nullable = true)
     |-- scope: string (nullable = true)
     |-- start_date: date (nullable = true)
     |-- end_date: date (nullable = true)
     |-- source_url: string (nullable = true)

In [ ]:
🔍 Preview da dimensão:
    +----------------------------------------------------------------+------------------------------------------+-------------------+-------------------------------------------------------------------+--------+----------+----------+-----------------------------+
    |sk_policy                                                       |policy_name                               |policy_type        |description                                                        |scope   |start_date|end_date  |source_url                   |
    +----------------------------------------------------------------+------------------------------------------+-------------------+-------------------------------------------------------------------+--------+----------+----------+-----------------------------+
    |fcd5b4d28d1e267faddf3e3b3ca82f886813b6d486b5dfc17d781318c8509bc8|Plano Nacional de Mobilidade Elétrica 2023|Mobilidade Elétrica|Plano para acelerar a instalação de postos de carregamento EV.     |Nacional|2023-01-01|2025-12-31|https://www.fundoambiental.pt|
    |8d6b66ba1ccc5da17fccf12f5fe5187cc4f428634eb1d0554fa36c27c7ddb0a6|Incentivo ao Autoconsumo 2022             |Energia Renovável  |Incentivo para instalações de produção fotovoltaica doméstica.     |Nacional|2022-04-01|2024-12-31|https://www.dgeg.gov.pt      |
    |e182f788feafe6f93dac5016ff45d88df5ee9def3dff67a9357999715a736803|Plano E-REDES Smart Metering              |Digitalização      |Plano para massificação de contadores inteligentes na rede.        |E-REDES |2021-01-01|2025-12-31|https://www.e-redes.pt       |
    |e7c3284a07784ac6412231e7f317d403c3600a3c98e65e6bccbab8276e495683|Injeção Nacional Renovável                |Energia Renovável  |Medida transversal de produção renovável injetada na rede nacional.|Nacional|2020-01-01|2025-12-31|https://www.erse.pt          |
    +----------------------------------------------------------------+------------------------------------------+-------------------+-------------------------------------------------------------------+--------+----------+----------+-----------------------------+

In [ ]:
💾 Saving as Delta in Gold layer...
    🗃️ Registering as SQL Table...
    ✅ SQL Table 'Dim_Policy' created and registered with success.

In [ ]:
dim_technology

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import sha2, col
    from pyspark.sql.types import StructType, StructField, StringType, BooleanType

In [ ]:
# Start Spark session
    spark = SparkSession.builder.getOrCreate()

In [ ]:
print("📥 Criar dim_technology...")

In [ ]:
# STEP 1 – Definir lista de tecnologias
    technologies = [
        {
            "technology_type": "Solar Fotovoltaica",
            "source": "Renovável",
            "description": "Conversão direta da luz solar em eletricidade.",
            "is_renewable": True
        },
        {
            "technology_type": "Eólica",
            "source": "Renovável",
            "description": "Geração de energia a partir do vento.",
            "is_renewable": True
        },
        {
            "technology_type": "Hídrica",
            "source": "Renovável",
            "description": "Energia gerada por barragens e correntes de água.",
            "is_renewable": True
        },
        {
            "technology_type": "Biomassa",
            "source": "Renovável",
            "description": "Conversão de matéria orgânica em energia.",
            "is_renewable": True
        },
        {
            "technology_type": "Cogeração",
            "source": "Mista",
            "description": "Produção simultânea de eletricidade e calor.",
            "is_renewable": False
        },
        {
            "technology_type": "Combustíveis Fósseis",
            "source": "Fóssil",
            "description": "Carvão, gás natural ou derivados do petróleo.",
            "is_renewable": False
        }
    ]

In [ ]:
schema = StructType([
        StructField("technology_type", StringType(), False),
        StructField("source", StringType(), False),
        StructField("description", StringType(), True),
        StructField("is_renewable", BooleanType(), True)
    ])

In [ ]:
# STEP 2 – Criar DataFrame e adicionar SK
    df_tech = (
        spark.createDataFrame(technologies, schema)
        .withColumn("sk_technology", sha2(col("technology_type"), 256))
        .select("sk_technology", "technology_type", "source", "description", "is_renewable")
    )

In [ ]:
# STEP 3 – Preview
    print("🔍 Preview da dim_technology:")
    df_tech.show(truncate=False)

In [ ]:
# STEP 4 – Guardar como Delta Table na camada Gold
    path_gold = "Files/gold/dim_technology.delta"
    print(f"💾 Guardar dim_technology como Delta em: {path_gold}")
    df_tech.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(path_gold)

In [ ]:
# STEP 5 – Registar como tabela SQL
    print("🗃️ Registar como SQL Table...")
    spark.sql("DROP TABLE IF EXISTS dim_technology")
    df_tech.write.mode("overwrite").format("delta").saveAsTable("dim_technology")

In [ ]:
print("✅ SQL Table 'dim_technology' criada com sucesso 🎯")

In [ ]:
{"execution_finish_time":"2025-06-27T23:27:51.5863524Z","execution_start_time":"2025-06-27T23:27:43.0025756Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"df7c761d-79c6-4350-9ccc-0c617317a063","queued_time":"2025-06-27T23:27:43.0012415Z","session_id":"62f3a26e-05ae-4dde-b7d2-bf31a7a8f0f0","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":9,"statement_ids":[9]}

In [ ]:
📥 Criar DimTechnology...
    🔍 Preview da DimTechnology:
    +----------------------------------------------------------------+--------------------+---------+-------------------------------------------------+------------+
    |sk_technology                                                   |technology_type     |source   |description                                      |is_renewable|
    +----------------------------------------------------------------+--------------------+---------+-------------------------------------------------+------------+
    |4e60407d375426868dd4c8d3bf48142e906dd5e11cf1328333ee195f9a641550|Solar Fotovoltaica  |Renovável|Conversão direta da luz solar em eletricidade.   |true        |
    |0be31780f5366cf93147741ef3f7d09c30269fa4541af88b555e9edf54e461b9|Eólica              |Renovável|Geração de energia a partir do vento.            |true        |
    |a017e3647e916a8ad0ebbde21a66bc96cbc45a58a1c532a36398df5d438a9bc3|Hídrica             |Renovável|Energia gerada por barragens e correntes de água.|true        |
    |d82cb98ae38b67daa1d854ac7b5ffa831b776956c9972af6818ebdc0312955ba|Biomassa            |Renovável|Conversão de matéria orgânica em energia.        |true        |
    |dab835bcd413d860b8cfb4a7d265f6f4ff67d9bb0810c07e552c5eaff1583257|Cogeração           |Mista    |Produção simultânea de eletricidade e calor.     |false       |
    |8f66f6856dca1369bd016292484b8e4b80ee3dee156b160a00de0bcb7f1076ff|Combustíveis Fósseis|Fóssil   |Carvão, gás natural ou derivados do petróleo.    |false       |
    +----------------------------------------------------------------+--------------------+---------+-------------------------------------------------+------------+

In [ ]:
💾 Guardar DimTechnology como Delta em: Files/gold/DimTechnology.delta
    🗃️ Registar como SQL Table...
    ✅ SQL Table 'DimTechnology' criada com sucesso 🎯

In [ ]:
fact_evimpact

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import (
        col, when, concat_ws, lit, to_date, date_format,
        sum as _sum, count
    )

In [ ]:
# Iniciar sessão Spark
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# ============================
    # 1. Load Silver Cleaned Dataset
    # ============================
    df_ev = spark.read.format("delta").load("Files/silver/postos_carregamento_ves_cleaned.delta")
    print("📥 Loaded cleaned Silver dataset: postos_carregamento_ves_cleaned")

In [ ]:
# ============================
    # 2. Create sk_date from 'trimestre'
    # ============================
    df_ev = df_ev.withColumn(
        "date",
        when(col("trimestre").contains("T1"), concat_ws("-", col("trimestre").substr(1, 4), lit("03"), lit("31")))
        .when(col("trimestre").contains("T2"), concat_ws("-", col("trimestre").substr(1, 4), lit("06"), lit("30")))
        .when(col("trimestre").contains("T3"), concat_ws("-", col("trimestre").substr(1, 4), lit("09"), lit("30")))
        .when(col("trimestre").contains("T4"), concat_ws("-", col("trimestre").substr(1, 4), lit("12"), lit("31")))
    )

In [ ]:
df_ev = df_ev.withColumn("date", to_date("date", "yyyy-MM-dd"))
    df_ev = df_ev.withColumn("sk_date", date_format("date", "yyyyMMdd").cast("int"))

In [ ]:
# ============================
    # 3. Cast potência to float
    # ============================
    df_ev = df_ev.withColumn("potencia_kw", col("potencia_maxima_admissivel").cast("float"))

In [ ]:
# ============================
    # 4. Aggregate
    # ============================
    df_fact = df_ev.groupBy("sk_date", "sk_municipality").agg(
        _sum("potencia_kw").alias("total_power_kw"),
        count("*").alias("total_ev_stations")
    )

In [ ]:
df_fact.show(10, truncate=False)

In [ ]:
# ============================
    # 5. Save and Register SQL Table
    # ============================
    table_name = "fact_evimpact"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
# Guardar em formato Delta (Gold Layer)
    df_fact.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .save(path_gold)
    print(f"💾 Saved {table_name} to Gold at {path_gold}")

In [ ]:
# Registar como SQL Table no metastore
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")

In [ ]:
df_fact.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable(table_name)
    print(f"✅ SQL Table '{table_name}' registada com sucesso no Fabric")

In [ ]:
{"execution_finish_time":"2025-06-24T20:22:28.945451Z","execution_start_time":"2025-06-24T20:22:16.7956379Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"57404efe-4577-410f-bdab-63e308caf782","queued_time":"2025-06-24T20:22:16.7943426Z","session_id":"d80b9515-da52-44ad-b2ce-38e88a008065","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":5,"statement_ids":[5]}

In [ ]:
📥 Loaded cleaned Silver dataset: postos_carregamento_ves_cleaned
    +--------+----------------------------------------------------------------+------------------+-----------------+
    |sk_date |sk_municipality                                                 |total_power_kw    |total_ev_stations|
    +--------+----------------------------------------------------------------+------------------+-----------------+
    |20240331|bda80c54f8bd3c03bc4f855de37cfd0297a46ce41a664fa56efea92b35a8a2c0|201.4000015258789 |2                |
    |20231231|a79128db844c02483d44e128263c81b0f3067a6ed14246eed583d345bdfdb406|2259.050002336502 |12               |
    |20250331|df5583e09f57f5b32c4ecb311c686d48d12761ea7bab100caea7b9b81a042109|209.3000030517578 |4                |
    |20230630|f700f777c1060f78ea151e7a57b5eec77bc57c7f0fd9d1bb6595bb9d3766e2d3|2994.400001525879 |8                |
    |20230331|4e42b48a241897a28255dea5f2d2af759ff6861423ecce8b482aa0fe63df448d|1791.400001525879 |5                |
    |20230331|d94ddc3dc7a9a2e97841c518361778856c6935212ad3112c1dde08c40104c0b7|1442.7000007629395|5                |
    |20231231|9621496898bf8bc7cd67d440d56c5e632bc29a0bcce8edd198f4459b17cdedfd|41.400001525878906|1                |
    |20240630|ecd0ecbc968ced6719647c367ed0bb91bd400d0fe6e9edc0bcc0442ee2aca8fc|6249.150007724762 |25               |
    |20240630|7cb65d5a505579db83a1f8132c4a0511a61fde09d1620f850022158427e2ad9b|1909.6000022888184|8                |
    |20240331|9683060a2304d108fdafbde951c7daf7b50bb0a5f229835e952c9609a6a5f86b|606.4000015258789 |3                |
    +--------+----------------------------------------------------------------+------------------+-----------------+
    only showing top 10 rows

In [ ]:
💾 Saved Fact_EVImpact to Gold at Files/gold/Fact_EVImpact.delta
    ✅ SQL Table 'Fact_EVImpact' registada com sucesso no Fabric

In [ ]:
fact_loadbalance

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import (
        col, to_date, date_format, sum as _sum,
        month, year, concat_ws, lpad
    )

In [ ]:
# Iniciar Spark session
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# =============================
    # 1. Load Cleaned Silver Datasets
    # =============================
    df_consumo = spark.read.format("delta").load("Files/silver/3_consumos_faturados_por_municipio-ultimos_10_anos_cleaned.delta")
    df_producao = spark.read.format("delta").load("Files/silver/energia_produzida_total_nacional_cleaned.delta")
    print("📥 Loaded Silver datasets for consumption and production.")

In [ ]:
# =============================
    # 2. Aggregate by Month
    # =============================
    df_consumo_agg = df_consumo.groupBy("ano", "mes", "sk_municipality").agg(
        _sum(col("energia_ativa_kwh").cast("double")).alias("total_consumption_kwh")
    ).withColumn(
        "sk_date", concat_ws("", col("ano"), lpad(col("mes").cast("string"), 2, "0")).cast("int")
    )

In [ ]:
df_producao_agg = df_producao \
        .withColumn("year", year(col("date"))) \
        .withColumn("month", month(col("date"))) \
        .groupBy("year", "month").agg(
            _sum(col("energy_production").cast("double")).alias("national_production_kwh")
        ).withColumn(
            "sk_date", concat_ws("", col("year"), lpad(col("month").cast("string"), 2, "0")).cast("int")
        ).drop("year", "month")

In [ ]:
# =============================
    # 3. Join and Calculate Net Load
    # =============================
    df_fact = df_consumo_agg.join(df_producao_agg, on="sk_date", how="left") \
        .withColumn("net_load_kwh", col("total_consumption_kwh") - col("national_production_kwh"))

In [ ]:
# =============================
    # 4. Preview
    # =============================
    df_fact.show(10, truncate=False)

In [ ]:
# =============================
    # 5. Save and Register SQL Table
    # =============================
    table_name = "fact_loadbalance"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
df_fact.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .save(path_gold)
    print(f"💾 Saved {table_name} to Gold path: {path_gold}")

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    df_fact.write \
        .option("overwriteSchema", "true") \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable(table_name)
    print(f"✅ SQL Table '{table_name}' registada com sucesso no Fabric.")

In [ ]:
{"execution_finish_time":"2025-06-24T20:27:27.707034Z","execution_start_time":"2025-06-24T20:27:15.7495099Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"9188ec80-bec0-4016-bee7-bb305e0bc165","queued_time":"2025-06-24T20:27:15.7482881Z","session_id":"d80b9515-da52-44ad-b2ce-38e88a008065","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":7,"statement_ids":[7]}

In [ ]:
📥 Loaded Silver datasets for consumption and production.
    +-------+----+---+----------------------------------------------------------------+---------------------+-----------------------+---------------------+
    |sk_date|ano |mes|sk_municipality                                                 |total_consumption_kwh|national_production_kwh|net_load_kwh         |
    +-------+----+---+----------------------------------------------------------------+---------------------+-----------------------+---------------------+
    |202401 |2024|1  |4d548909d1f046f8484a243213d2b0dd983660408fb0690cd5edc634941db7e2|8786070.81           |4.913696077E9          |-4.90491000619E9     |
    |202401 |2024|1  |24e9ceba453a0004de633b3adcd7a6c845dc4b243243f8d60c7493ca868b1641|3373563.501          |4.913696077E9          |-4.910322513499E9    |
    |202401 |2024|1  |da13e83a4e76108c71058ea6a661a8f2e6cc0a744c5e8bfbe3b3e04653302100|1894092.4900000002   |4.913696077E9          |-4.91180198451E9     |
    |202401 |2024|1  |f61848ec916e48d5fb3025c5d4a71a5830f00918f5378e8772720a3ab369b092|2829257.419          |4.913696077E9          |-4.910866819581E9    |
    |202401 |2024|1  |749458f6c8c6f44b275a4797926f4bbea7dd87cb4c86250e816c84dc230e0028|2107630.591          |4.913696077E9          |-4.911588446409E9    |
    |202402 |2024|2  |f066bea954388defb97834bc225b600b9cb2bfb9d06d48d31782bd5e4881d922|2707058.1550000003   |4.281320084732998E9    |-4.2786130265779977E9|
    |202310 |2023|10 |749458f6c8c6f44b275a4797926f4bbea7dd87cb4c86250e816c84dc230e0028|1733308.9419999998   |4.19074189725E9        |-4.189008588308E9    |
    |202311 |2023|11 |24e9ceba453a0004de633b3adcd7a6c845dc4b243243f8d60c7493ca868b1641|2842557.552          |4.3313461261500025E9   |-4.328503568598002E9 |
    |202312 |2023|12 |2c9ab75b237f76b170cabfabfb7693bb4dd5d149e5092f3f2504166297eae136|1.2926778182E7       |4.72736664735E9        |-4.714439869168E9    |
    |202301 |2023|1  |8d24b48f7084913dd59bc8d6001b71f3c194122cf0dbab5ae46e7e28e1bf787a|1685712.2189999998   |4.83476878725E9        |-4.833083075031E9    |
    +-------+----+---+----------------------------------------------------------------+---------------------+-----------------------+---------------------+
    only showing top 10 rows

In [ ]:
💾 Saved Fact_LoadBalance to Gold path: Files/gold/Fact_LoadBalance.delta
    ✅ SQL Table 'Fact_LoadBalance' registada com sucesso no Fabric.

In [ ]:
fact_renewablegoals

In [ ]:
from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, sha2

In [ ]:
# Iniciar Spark session
    spark = SparkSession.builder.getOrCreate()

In [ ]:
# =============================
    # 1. Criar Dimensão de Metas Energéticas
    # =============================
    energy_targets_data = [
        (2026, 80),
        (2027, 82),
        (2028, 83),
        (2029, 84),
        (2030, 85),
        (2031, 85),
        (2032, 85),
        (2033, 85),
        (2034, 85),
        (2035, 85),
    ]

In [ ]:
columns = ["year", "target_renewable_pct"]
    df_energy_target = spark.createDataFrame(energy_targets_data, columns)

In [ ]:
# Adicionar surrogate key
    df_energy_target = df_energy_target.withColumn(
        "sk_energy_target", sha2(col("year").cast("string"), 256)
    )

In [ ]:
# =============================
    # 2. Guardar e Registar Tabela SQL
    # =============================
    table_name = "fact_renewablegoals"
    path_gold = f"Files/gold/{table_name}.delta"

In [ ]:
df_energy_target.write.option("overwriteSchema", "true").mode("overwrite").format("delta").save(path_gold)
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
    df_energy_target.write.option("overwriteSchema", "true").mode("overwrite").format("delta").saveAsTable(table_name)

In [ ]:
{"execution_finish_time":"2025-07-14T20:27:50.2082451Z","execution_start_time":"2025-07-14T20:27:22.3872334Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"28f7934e-da8c-4b49-b0dc-00228c8c65ae","queued_time":"2025-07-14T20:27:12.4397938Z","session_id":"50864d74-6a26-4450-ae18-9556e9e52e4e","session_start_time":"2025-07-14T20:27:12.4407509Z","spark_pool":null,"state":"finished","statement_id":3,"statement_ids":[3]}

In [ ]:
dim_codconcelho (to add the codconcelho later in dim_municipality)

In [ ]:
from pyspark.sql.functions import col, sha2, concat_ws, trim, upper, lpad

In [ ]:
bronze_path = "Files/bronze/cod_concelho"

In [ ]:
df_raw = (
        spark.read.option("header", True)
        .option("inferSchema", False)  # IMPORTANTE para não remover zeros
        .csv(bronze_path)
    )

In [ ]:
df_clean = (
        df_raw
        .select(
            trim(col("con_code")).alias("cod_concelho"),
            trim(col("con_name")).alias("municipality"),
            trim(col("dis_name")).alias("district")
        )
        .dropna(subset=["cod_concelho", "municipality", "district"])
        .withColumn("municipality", upper(trim(col("municipality"))))
        .withColumn("district", upper(trim(col("district"))))
        .dropDuplicates(["cod_concelho"])
    )

In [ ]:
# Garantir que cod_concelho tem sempre 4 dígitos
    df_clean = df_clean.withColumn("cod_concelho", lpad(col("cod_concelho"), 4, "0"))

In [ ]:
df_final = df_clean.withColumn(
        "sk_municipality",
        sha2(concat_ws("-", col("district"), col("municipality")), 256)
    )

In [ ]:
# Preview atualizado
    print("✅ Preview da dim_codconcelho (corrigido com 4 dígitos):")
    df_final.show(10, truncate=False)

In [ ]:
# Guardar no Silver
    df_final.write.mode("overwrite").format("delta").save("Files/silver/dim_codconcelho.delta")
    df_final.write.mode("overwrite").format("delta").saveAsTable("dim_codconcelho")

In [ ]:
{"execution_finish_time":"2025-06-28T13:42:55.0397748Z","execution_start_time":"2025-06-28T13:42:45.0853984Z","livy_statement_state":"available","normalized_state":"finished","parent_msg_id":"24ee0698-1b64-416f-89bc-40da2fd485b7","queued_time":"2025-06-28T13:42:45.0841216Z","session_id":"ccfaf4ac-d020-4f44-a99a-588f8e59ebfd","session_start_time":null,"spark_pool":null,"state":"finished","statement_id":11,"statement_ids":[11]}

In [ ]:
✅ Preview da DimCodConcelho (corrigido com 4 dígitos):
    +------------+-------------------+--------+----------------------------------------------------------------+
    |cod_concelho|municipality       |district|sk_municipality                                                 |
    +------------+-------------------+--------+----------------------------------------------------------------+
    |1001        |ALCOBAÇA           |LEIRIA  |00ed80ce99753ed4d66f64d1ff3a9faad0fb2f23f5a2f934a98f2923083b9995|
    |1002        |ALVAIÁZERE         |LEIRIA  |2ac42b0093cbdf4d196fa28520bc1bfb5c3cc3e1975d6843dbb16eb8b42e0bfc|
    |1003        |ANSIÃO             |LEIRIA  |03449b54a14c4ab8c119fe03278c208dc98f861fa43b76151c0c2a17f4fa7cc1|
    |1004        |BATALHA            |LEIRIA  |477cb542f563b368192f7b9cbe7d8ea2dc2c2fa549477b607b1da31265184757|
    |1005        |BOMBARRAL          |LEIRIA  |3d2813e1b94509efd2a51a29a22286a761933b32b957e2e3d81807f7d79d786d|
    |1006        |CALDAS DA RAINHA   |LEIRIA  |2c9ab75b237f76b170cabfabfb7693bb4dd5d149e5092f3f2504166297eae136|
    |1007        |CASTANHEIRA DE PÊRA|LEIRIA  |36402fad64f4dd5e1c76560e78329e91af2619adbbf70e7907bdb9e491f1f7e6|
    |1008        |FIGUEIRÓ DOS VINHOS|LEIRIA  |a610f919e2edaf37e277729af04334dc574279f71a2f3995b607b1f10af7e200|
    |1009        |LEIRIA             |LEIRIA  |c2f60e07f6b618749f21a498803929582f2aa416685c1257641a572142fa65a5|
    |0101        |ÁGUEDA             |AVEIRO  |3b9f26ab28a1892f70d247bcd5287927a7fd8fb0540ad98f3facad472933c77b|
    +------------+-------------------+--------+----------------------------------------------------------------+
    only showing top 10 rows